# Zindi User Behaviour-


## Description: Zindi is an African Data Science Platform with an ecosystem of both data scientists and companies requiring data science solutions in form of prized challenges. Zindi has garnered users who interact with the platform and make contributions towards enhancing the data science frontier. 

## Users participate on Zindi in form of competition participations, comments and discussion contributions, the consistency of participation varies across users. In order to properly plan and anticipate attrition early enough, Zindi needs to predict user behaviour for the following months given historical user behaviour about their individual specific interactions on the platform. An active user is one that either makes a submission, participates in a competition, makes a discussion or comment


## There are dataframes given on:

1) User and Competition characteristics 

2) submissions, discussions, comments, competition participations for each user 

3) User behaviour for the year and month showing active status.

## The objective is to extract historical information about users and competition relationships interaction in order to predict future interaction status.

## This is a Data Preparation Notebook with steps outlined-

1) Data importing and Overview

2) EDA- Understanding the given datasets and structure

3) Features Extraction for modelling:

    - Statistics about on-going competitions (Nature of competitions, characteristics of the competitions)
    
    - User historical behaviour extracted from all available database up till timeline of prediction. (Historical user behaviour or Observed user behaviour)
    
    - User information and characterisitics available on the platform
    
    - User activity from the previous month
    
    - User activity(Specific level detail) in the last 3 months
    
    - User activity(Specific level detail) in the last 6 months

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from tqdm import tqdm
import seaborn as sns
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.patches as mpatches
from pandas.tseries.offsets import DateOffset
from dateutil import relativedelta
import re
import random
rand = 40
np.random.seed(rand)
random.seed(rand)
tqdm.pandas()
import logging
logging.basicConfig(level='INFO')
import os

In [2]:
#import datasets
train = pd.read_csv('./data/Train.csv')
test = pd.read_csv('./data/Test.csv')
users = pd.read_csv('./data/Users.csv')
submissions = pd.read_csv('./data/Submissions.csv')
comments = pd.read_csv('./data/Comments.csv')
competitions = pd.read_csv('./data/Competitions.csv')
discussions = pd.read_csv('./data/Discussions.csv')
compPart = pd.read_csv('./data/CompetitionPartipation.csv')
vardef = pd.read_csv('./data/VariableDefinitions.csv')
samplesub = pd.read_csv('./data/SampleSubmission.csv')

In [3]:
users

,UserID,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_N5LTBAPU,0,ID_DMRM,1,group 3,2,4,4
1,ID_CLSFQB0S,0,ID_Q02,3,group 3,1,5,4
2,ID_RE6T58Y4,0,ID_Q02,0,group 3,2,12,3
3,ID_XJQQRJV3,0,ID_Z8BI,0,group 3,2,9,2
4,ID_1JHU6A8S,0,ID_Q02,3,group 3,2,10,1
...,...,...,...,...,...,...,...,...
22402,ID_D4SARSC7,0,ID_5OWN,1,group 3,1,5,3
22403,ID_B8VJJMWK,0,ID_Q02,3,group 3,2,3,4
22404,ID_XAQGPGAZ,0,ID_Q02,3,group 3,2,3,1
22405,ID_1AO7PVP2,0,ID_Q02,3,group 3,2,5,5


## EDA-

### DATA OVERVIEW

**Understanding the meaning of variables: Reference at each step for variable understanding**

In [4]:
vardef.head()

,VariableName,VariableDefinition
0,Train,NaN
1,UserID,This is the unique ID of the user who made the...
2,CompPart,Value 1 if Participated in any competition in ...
3,Comment,Value 1 if Participated in any comments in the...
4,Sub,Value 1 if Submitted to any competition in the...


In [5]:
vardef[vardef['VariableName'] == 'UserID']['VariableDefinition'].values

array(['This is the unique ID of the user who made the comment',
       'This is the unique ID of the user who made the comment',
       'This is the unique ID of the user who enrolled in a specific CompID',
       'The user who created the discussion, this can be merged with the User table on UserID',
       'This is the unique ID of the user who made a submission',
       'Unique user ID, this feature can be used to merge onto the discussion, comments and submission table'],
      dtype=object)

**Train**

In [6]:
train.head()

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
0,ID_XI7BAR4Y,8,3,0,0,0,0,0
1,ID_XI7BAR4Y,8,2,0,0,0,0,0
2,ID_XI7BAR4Y,9,2,0,0,0,0,0
3,ID_XI7BAR4Y,9,3,0,0,0,0,0
4,ID_XI7BAR4Y,10,3,0,0,0,0,0


In [7]:
train.tail()

,User_ID,month,year,CompPart,Comment,Sub,Disc,Target
259827,ID_MAP5X6D4,12,3,0,0,0,0,0
259828,ID_QHUAHU76,12,3,0,0,0,0,0
259829,ID_8IKU22O5,12,3,1,0,0,0,1
259830,ID_NHWCR1IY,12,3,0,0,0,0,0
259831,ID_XMD7EIYV,12,3,1,0,0,1,1


In [8]:
#view the data types in the train data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259832 entries, 0 to 259831
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   User_ID   259832 non-null  object
 1   month     259832 non-null  int64 
 2   year      259832 non-null  int64 
 3   CompPart  259832 non-null  int64 
 4   Comment   259832 non-null  int64 
 5   Sub       259832 non-null  int64 
 6   Disc      259832 non-null  int64 
 7   Target    259832 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 15.9+ MB


In [9]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
month,259832.0,7.412116,3.355127,1.0,5.0,8.0,10.0,12.0
year,259832.0,2.690927,0.527094,1.0,2.0,3.0,3.0,3.0
CompPart,259832.0,0.120867,0.325973,0.0,0.0,0.0,0.0,1.0
Comment,259832.0,0.014921,0.121238,0.0,0.0,0.0,0.0,1.0
Sub,259832.0,0.064676,0.245954,0.0,0.0,0.0,0.0,1.0
Disc,259832.0,0.014532,0.119672,0.0,0.0,0.0,0.0,1.0
Target,259832.0,0.139729,0.346706,0.0,0.0,0.0,0.0,1.0


**Test**

In [10]:
test.head()

,User_ID,month,year
0,ID_H1ELY25E,1,4
1,ID_H1ELY25E,2,4
2,ID_H1ELY25E,3,4
3,ID_463Q2BCO,1,4
4,ID_463Q2BCO,2,4


In [11]:
test.tail()

,User_ID,month,year
65218,ID_4XKWR8UN,3,4
65219,ID_L54061S5,3,4
65220,ID_I3C1N5RO,3,4
65221,ID_WWNR6I15,3,4
65222,ID_V69HAZHO,3,4


In [12]:
test.shape

(65223, 3)

In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65223 entries, 0 to 65222
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   User_ID  65223 non-null  object
 1   month    65223 non-null  int64 
 2   year     65223 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [14]:
test.describe()

,month,year
count,65223.000000,65223.0
mean,2.022415,4.0
std,0.814879,0.0
min,1.000000,4.0
25%,1.000000,4.0
50%,2.000000,4.0
75%,3.000000,4.0
max,3.000000,4.0


**Users**

In [15]:
users.head()

,UserID,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_N5LTBAPU,0,ID_DMRM,1,group 3,2,4,4
1,ID_CLSFQB0S,0,ID_Q02,3,group 3,1,5,4
2,ID_RE6T58Y4,0,ID_Q02,0,group 3,2,12,3
3,ID_XJQQRJV3,0,ID_Z8BI,0,group 3,2,9,2
4,ID_1JHU6A8S,0,ID_Q02,3,group 3,2,10,1


In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22407 entries, 0 to 22406
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   UserID                22407 non-null  object
 1   FeatureX              22407 non-null  int64 
 2   Country               22407 non-null  object
 3   FeatureY              22407 non-null  int64 
 4   Points                22407 non-null  object
 5   UserDate Year         22407 non-null  int64 
 6   UserDate Month        22407 non-null  int64 
 7   UserDate Day_of_week  22407 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 1.4+ MB


In [17]:
users.describe()

,FeatureX,FeatureY,UserDate Year,UserDate Month,UserDate Day_of_week
count,22407.000000,22407.000000,22407.000000,22407.000000,22407.000000
mean,0.080332,0.677154,2.586290,6.451868,3.796448
std,0.271813,1.080417,0.783856,3.689670,1.985381
min,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,2.000000,3.000000,2.000000
50%,0.000000,0.000000,3.000000,6.000000,4.000000
75%,0.000000,1.000000,3.000000,10.000000,5.000000
max,1.000000,3.000000,4.000000,12.000000,7.000000


**Submissions**

In [18]:
submissions.head()

,UserID,FeatureG,CompID,SubDate Year,SubDate Month,SubDate Day_of_week
0,ID_8JP75F20,1,ID_GFDE,3,3,2
1,ID_8JP75F20,1,ID_GFDE,3,3,2
2,ID_8JP75F20,1,ID_GFDE,3,3,4
3,ID_8JP75F20,1,ID_GFDE,3,3,2
4,ID_8JP75F20,1,ID_GFDE,3,3,4


In [19]:
submissions.tail()

,UserID,FeatureG,CompID,SubDate Year,SubDate Month,SubDate Day_of_week
375758,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375759,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375760,ID_CX5N3Q88,1,ID_EZD0,3,11,3
375761,ID_J6MM98N2,1,ID_92AG,3,12,4
375762,ID_J6MM98N2,1,ID_92AG,3,12,4


In [20]:
submissions.describe()

,FeatureG,SubDate Year,SubDate Month,SubDate Day_of_week
count,375763.000000,375763.000000,375763.000000,375763.000000
mean,0.951858,2.450092,7.127599,4.203471
std,0.222336,0.618930,3.443989,1.879810
min,0.000000,1.000000,1.000000,1.000000
25%,1.000000,2.000000,4.000000,3.000000
50%,1.000000,3.000000,7.000000,5.000000
75%,1.000000,3.000000,11.000000,6.000000
max,3.000000,3.000000,12.000000,7.000000


**Comments**

In [21]:
comments.head()

,UserID,CommentDate Year,CommentDate Month,CommentDate Day_of_week
0,ID_MVIB05DL,3,1,7
1,ID_MVIB05DL,3,1,7
2,ID_KBRFRAR9,3,1,7
3,ID_UDS6FRR8,3,1,7
4,ID_UDS6FRR8,3,1,7


In [22]:
comments.tail()

,UserID,CommentDate Year,CommentDate Month,CommentDate Day_of_week
11746,ID_9UP7X8IA,3,5,7
11747,ID_9MJ12JJR,3,5,4
11748,ID_0B9SK73T,3,6,5
11749,ID_0B9SK73T,3,6,4
11750,ID_2IKJFHYK,3,6,1


In [23]:
comments.describe()

,CommentDate Year,CommentDate Month,CommentDate Day_of_week
count,11751.000000,11751.000000,11751.000000
mean,2.626585,6.371288,4.016254
std,0.551320,3.608655,2.028728
min,1.000000,1.000000,1.000000
25%,2.000000,3.000000,2.000000
50%,3.000000,5.000000,4.000000
75%,3.000000,10.000000,6.000000
max,3.000000,12.000000,7.000000


**Competitions**

In [24]:
competitions.head()

,CompID,Country,FeatureA,FeatureB,FeatureC,Kind,Points Reward,FeatureD,SecretCode,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week
0,ID_WGZ2,ID_HWRH,[1],[14],1.0,1,27,1,0,100.0,[1],1,8.0,7.0,1,6,5
1,ID_G370,NaN,[1],[],2.0,1,52,2,0,10.0,[1],4,1.0,6.0,3,12,7
2,ID_R5HL,NaN,[1],[],3.0,1,126,2,0,10.0,[1],4,1.0,6.0,3,10,4
3,ID_Y6XI,ID_ARVG,[1],[],4.0,1,52,2,0,10.0,[1],4,1.0,6.0,3,11,4
4,ID_8PEN,ID_I1L9,[1],[],5.0,0,2,2,0,200.0,[1],2,11.0,6.0,2,11,4


In [25]:
competitions['FeatureD'].value_counts()

2    96
3    48
1    10
Name: FeatureD, dtype: int64

In [26]:
competitions.tail()

,CompID,Country,FeatureA,FeatureB,FeatureC,Kind,Points Reward,FeatureD,SecretCode,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week
149,ID_F7X4,NaN,[1],[],2.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,1,5,3
150,ID_E1LI,ID_I1L9,"[3, 2]",[14],7.0,1,2,3,0,50.0,[2],not mapped,NaN,NaN,3,2,3
151,ID_MPSN,NaN,[1],[9],2.0,1,2,3,0,10.0,"[1, 5]",not mapped,NaN,NaN,2,2,7
152,ID_Z5QP,ID_5OWN,[1],[],31.0,0,2,2,1,NaN,[1],3,6.0,5.0,3,6,5
153,ID_D42Y,ID_ARVG,[1],[],2.0,1,151,1,0,30.0,[1],2,7.0,7.0,2,3,4


In [27]:
competitions.describe()

,FeatureC,Kind,Points Reward,FeatureD,SecretCode,SubmissionLimitPerDay,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week
count,137.000000,154.000000,154.000000,154.000000,154.000000,150.00000,134.000000,134.000000,154.000000,154.000000,154.000000
mean,11.934307,0.500000,72.987013,2.246753,0.396104,46.56000,6.074627,5.388060,2.675325,5.987013,4.493506
std,10.393864,0.501631,126.295172,0.563737,0.490682,51.37211,3.808631,1.481264,0.774454,3.785053,1.576986
min,1.000000,0.000000,2.000000,1.000000,0.000000,3.00000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.000000,0.000000,2.000000,2.000000,0.000000,10.00000,3.000000,5.000000,2.000000,3.000000,4.000000
50%,9.000000,0.500000,2.000000,2.000000,0.000000,30.00000,5.500000,6.000000,3.000000,5.000000,4.000000
75%,18.000000,1.000000,102.000000,3.000000,1.000000,50.00000,10.000000,6.000000,3.000000,10.000000,5.000000
max,37.000000,1.000000,599.000000,3.000000,1.000000,200.00000,12.000000,7.000000,4.000000,12.000000,7.000000


**Discussions**

In [28]:
discussions.head()

,FeatureF,DiscDate Year,DiscDate Month,DiscDate Day_of_week,DiscID,UserID
0,1,3,10,1,ID_Z77ETQ,ID_F2757IAI
1,0,3,12,3,ID_E47JKY,ID_F2757IAI
2,1,3,1,3,ID_CB4Y0N,ID_F2757IAI
3,0,3,9,7,ID_BNIHCF,ID_F2757IAI
4,1,3,5,7,ID_MLPYCO,ID_F2757IAI


In [29]:
discussions.tail()

,FeatureF,DiscDate Year,DiscDate Month,DiscDate Day_of_week,DiscID,UserID
6206,0,3,7,1,ID_07HHT5,ID_E2Q1K4TQ
6207,1,3,7,2,ID_9TID7A,ID_8I5VPQIF
6208,0,3,7,5,ID_IMGAT1,ID_UC2B2DBT
6209,1,3,11,2,ID_W3CY00,ID_VVUWHX7W
6210,0,3,11,3,ID_GT26RF,ID_A9FRILEL


In [30]:
discussions.describe()

,FeatureF,DiscDate Year,DiscDate Month,DiscDate Day_of_week
count,6211.000000,6211.000000,6211.000000,6211.000000
mean,0.511029,2.759620,6.352922,4.137176
std,0.499919,0.484562,3.476361,1.929645
min,0.000000,1.000000,1.000000,1.000000
25%,0.000000,3.000000,3.000000,3.000000
50%,1.000000,3.000000,5.000000,4.000000
75%,1.000000,3.000000,10.000000,6.000000
max,1.000000,3.000000,12.000000,7.000000


**CompPart**

In [31]:
compPart.head(20)

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
0,ID_WMUF,ID_UWBBZ9OF,rank 11,count 10,3,1,4
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7
2,ID_WMUF,ID_1N5J2PGO,NaN,count 10,3,1,5
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4
5,ID_50DK,ID_1N5J2PGO,NaN,NaN,2,11,7
6,ID_MPSN,ID_1N5J2PGO,NaN,NaN,2,10,6
7,ID_AWEI,ID_1N5J2PGO,rank 6,count 10,2,4,1
8,ID_ON5E,ID_1N5J2PGO,NaN,NaN,3,9,4
9,ID_QQTZ,ID_1N5J2PGO,NaN,NaN,1,12,2


In [32]:
compPart.tail()

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
48560,ID_XYJZ,ID_5C4D0VO2,rank 11,count 10,3,7,6
48561,ID_XYJZ,ID_JRJZQB8S,rank 11,count 10,3,7,5
48562,ID_XYJZ,ID_H4FM7RBV,NaN,NaN,3,7,5
48563,ID_XYJZ,ID_C3QOEMU4,rank 11,count 10,3,7,5
48564,ID_XYJZ,ID_WF3E1TND,NaN,NaN,3,7,7


In [33]:
compPart.describe()

,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
count,48565.000000,48565.000000,48565.000000
mean,2.505487,7.054216,3.942098
std,0.596122,3.542464,1.899356
min,1.000000,1.000000,1.000000
25%,2.000000,4.000000,2.000000
50%,3.000000,7.000000,4.000000
75%,3.000000,10.000000,5.000000
max,3.000000,12.000000,7.000000


In [34]:
#Replace to numbers to enable feature engineering
compPart['PublicRank'] = compPart['PublicRank'].str.replace('rank', '')
compPart['Successful Submission Count'] = compPart['Successful Submission Count'].str.replace('count', "")
compPart['PublicRank'] = compPart['PublicRank'].astype(float)
compPart['Successful Submission Count'] = compPart['Successful Submission Count'].astype(float)

In [35]:
compPart.head()

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week
0,ID_WMUF,ID_UWBBZ9OF,11.0,10.0,3,1,4
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7
2,ID_WMUF,ID_1N5J2PGO,NaN,10.0,3,1,5
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4


**SampleSubmission**

In [37]:
samplesub.head()

,UserMonthYear,Target
0,ID_000VV0KM_1_4,NaN
1,ID_000VV0KM_2_4,NaN
2,ID_000VV0KM_3_4,NaN
3,ID_003OCIYO_1_4,NaN
4,ID_003OCIYO_2_4,NaN


In [38]:
samplesub.tail()

,UserMonthYear,Target
65218,ID_ZZVPF22K_2_4,NaN
65219,ID_ZZVPF22K_3_4,NaN
65220,ID_ZZXDLYXB_1_4,NaN
65221,ID_ZZXDLYXB_2_4,NaN
65222,ID_ZZXDLYXB_3_4,NaN


In [39]:
samplesub.describe()

,Target
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


Observation- The dataset is ordered in a time series nature, dataset needs to be organized in a time series nature, so the information in a later timeline shouldn't be leaked into an earlier timeline in evaluation

### DATA STRUCTURE UNDERSTANDING

**Understanding how the user dataframe is organized-**

In [40]:
users.head()

,UserID,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_N5LTBAPU,0,ID_DMRM,1,group 3,2,4,4
1,ID_CLSFQB0S,0,ID_Q02,3,group 3,1,5,4
2,ID_RE6T58Y4,0,ID_Q02,0,group 3,2,12,3
3,ID_XJQQRJV3,0,ID_Z8BI,0,group 3,2,9,2
4,ID_1JHU6A8S,0,ID_Q02,3,group 3,2,10,1


In [41]:
users.columns

Index(['UserID', 'FeatureX', 'Country', 'FeatureY', 'Points', 'UserDate Year',
       'UserDate Month', 'UserDate Day_of_week'],
      dtype='object')

In [42]:
#select users that are in train set
users_train = users[users['UserID'].isin(list(train['User_ID'].unique()))]

len(users_train)


20218

In [43]:
#all users in train set are accounted for
len(train['User_ID'].unique())

20218

In [44]:
users_train

,UserID,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_N5LTBAPU,0,ID_DMRM,1,group 3,2,4,4
1,ID_CLSFQB0S,0,ID_Q02,3,group 3,1,5,4
2,ID_RE6T58Y4,0,ID_Q02,0,group 3,2,12,3
3,ID_XJQQRJV3,0,ID_Z8BI,0,group 3,2,9,2
4,ID_1JHU6A8S,0,ID_Q02,3,group 3,2,10,1
...,...,...,...,...,...,...,...,...
22402,ID_D4SARSC7,0,ID_5OWN,1,group 3,1,5,3
22403,ID_B8VJJMWK,0,ID_Q02,3,group 3,2,3,4
22404,ID_XAQGPGAZ,0,ID_Q02,3,group 3,2,3,1
22405,ID_1AO7PVP2,0,ID_Q02,3,group 3,2,5,5


**Confirming users train & test intersection: Are all users in test set already in train set**

The challenge involves learning about historical user behaviour in train datasets and applying the model to predict the outcome of the user in a later timeline. We need to ensure all users in test set are in train set

In [45]:
#check users train & test intersect
len(set(test['User_ID']).intersection(train['User_ID']))/len(set(test['User_ID']))

0.9025087045799483

90% of test users have some historical information in train set

In [46]:
non_train = set(test['User_ID']) - set(test['User_ID']).intersection(set(train['User_ID']))
len(non_train)

2184

In [47]:
non_train

{'ID_AG55WSRI',
 'ID_1SLQB1ZF',
 'ID_AX0HIRJZ',
 'ID_F9I0LFCY',
 'ID_4VRSDZ3K',
 'ID_H1ELY25E',
 'ID_HZ8YGTCO',
 'ID_8UD7LPR9',
 'ID_MSBMGFNJ',
 'ID_IJTVIQON',
 'ID_L24MPIZX',
 'ID_GIB1NJQY',
 'ID_009C2VCA',
 'ID_1ZWVKN04',
 'ID_C03QGKP5',
 'ID_M7JXQ2XA',
 'ID_RKKK4197',
 'ID_OJYYOU1Q',
 'ID_ETF0G3L9',
 'ID_MZKQWGAK',
 'ID_J031GZJA',
 'ID_K7XO5UJI',
 'ID_GT8I8WU9',
 'ID_QTT37EJ1',
 'ID_GH1ZIT1M',
 'ID_ITDTBKLX',
 'ID_AO4G2X0W',
 'ID_MRQVV3QO',
 'ID_GDB93OXC',
 'ID_8855P4IA',
 'ID_2YKCREFN',
 'ID_UY2NQ0KF',
 'ID_WHYHFMNF',
 'ID_OIX135FM',
 'ID_RAK3LHBX',
 'ID_25TZPVQL',
 'ID_BADVTPKG',
 'ID_RD97625N',
 'ID_C489F3NO',
 'ID_R124M4CP',
 'ID_4V5NRBYT',
 'ID_434PMYD1',
 'ID_7WN2Z84A',
 'ID_MNCQSHTJ',
 'ID_8QID4O0K',
 'ID_XYJRDH7A',
 'ID_E70TRXY2',
 'ID_D9IWIR76',
 'ID_GWI1LZ6O',
 'ID_YVPN7LIA',
 'ID_KIVPTAHX',
 'ID_ZU2GRT4U',
 'ID_LLOY07LS',
 'ID_K4FEISMF',
 'ID_XJ0DWK0G',
 'ID_Z9RNPW5R',
 'ID_0HBX5H4S',
 'ID_1B52DF9C',
 'ID_RVD8WS6V',
 'ID_FTJH9XK9',
 'ID_ZKZSSOL6',
 'ID_LYL5OI0R',
 'ID_YCS

In [48]:
test[test['User_ID'].isin(list(non_train))]

,User_ID,month,year
0,ID_H1ELY25E,1,4
1,ID_H1ELY25E,2,4
2,ID_H1ELY25E,3,4
87,ID_7KM49VH9,1,4
88,ID_7KM49VH9,2,4
...,...,...,...
65218,ID_4XKWR8UN,3,4
65219,ID_L54061S5,3,4
65220,ID_I3C1N5RO,3,4
65221,ID_WWNR6I15,3,4


There are 2184 users not in training set, hence we won't have user information on their behaviour, which will affect our performance on 4569 rows. We check users datatframe if there is information on these users

In [49]:
len(non_train.intersection(users['UserID']))

2184

**Confirm what dates users not in train set in dataframe joined?**

In [50]:
users[users['UserID'].isin(list(non_train))]['UserDate Year'].value_counts()

4    2184
Name: UserDate Year, dtype: int64

All the users with no training history in training dataset is because they only joined the platform at the test timeline period, we can only try and apply information learned from similar users to predict this new users. We can consider other ways to handle this user behaviour later

In [51]:
#merge train and test for joint data analysis

train['tag'] = 'train'
test['tag'] = 'test'

all_data = pd.concat([train, test])

In [52]:
all_data = all_data.rename(columns = {'User_ID' : 'UserID'})

In [53]:
#merge user data to train/test
all_data = all_data.merge(users, on = 'UserID', how = 'left')

In [54]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,Country,FeatureY,Points,UserDate Year,UserDate Month,UserDate Day_of_week
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,ID_GU74,0,group 3,2,2,1
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,ID_GU74,0,group 3,2,2,1
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,ID_GU74,0,group 3,2,2,1
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,ID_GU74,0,group 3,2,2,1
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,ID_GU74,0,group 3,2,2,1


### FEATURE EXTRACTION

### Data Preparation Approach: Building the dataset by only adding overall historical feature statistics about supplementary datasets and historical user activity status from training dataset related to ongoing competitions and corresponding user response-

**Statistics are extracted in 3 ways- From overall user data, previous month, 3months data- Overall data represents holistic user behaviour, previous month represents most recent user behaviour, 3 & 6 months takes into account most recent time period of user behaviour**

### 1) Competitions Profile/database at the exact timeline of prediction

**Certain competitions are knowledge competitions without any end date, we can replace not mapped with np.nan**

In [55]:
competitions[competitions['CompEndTime Year'] == 'not mapped']

,CompID,Country,FeatureA,FeatureB,FeatureC,Kind,Points Reward,FeatureD,SecretCode,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week
6,ID_GOXL,ID_I1L9,"[2, 3]",[],6.0,1,2,3,0,30.0,[2],not mapped,NaN,NaN,3,1,4
7,ID_88GO,ID_GU74,[1],[],5.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,3,1,4
8,ID_HTKR,ID_CKI7,"[3, 2]",[],6.0,1,2,3,0,30.0,[2],not mapped,NaN,NaN,3,2,3
18,ID_P8IG,ID_CXB1,[1],[],8.0,1,2,3,0,30.0,[2],not mapped,NaN,NaN,2,4,4
23,ID_GCU7,ID_5OWN,[1],[],11.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,2,6,1
26,ID_LMFN,ID_5OWN,[1],[],6.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,1,5,3
28,ID_HJ9S,ID_CXB1,[1],[],13.0,1,2,3,0,30.0,[1],not mapped,NaN,NaN,2,4,4
29,ID_RSV3,ID_CXB1,"[4, 2]",[],14.0,1,2,3,0,30.0,[3],not mapped,NaN,NaN,2,4,4
45,ID_TLNA,NaN,"[2, 4]",[7],9.0,1,2,3,0,30.0,"[3, 5]",not mapped,NaN,NaN,2,3,4
66,ID_VDSL,ID_I1L9,"[2, 4]",[14],15.0,1,2,3,0,30.0,[3],not mapped,NaN,NaN,2,12,4


In [56]:
competitions['CompEndTime Year'] = competitions['CompEndTime Year'].replace('not mapped', np.nan)

In [57]:
for col in [['CompEndTime Year', 'CompStartTime Year', 'CompEndTime Month', 'CompStartTime Month']]:
    
    competitions[col] = competitions[col].astype(float)

In [58]:
competitions['CompEndTime Year'].max()

4.0

In [59]:
competitions.columns

Index(['CompID', 'Country', 'FeatureA', 'FeatureB', 'FeatureC', 'Kind',
       'Points Reward', 'FeatureD', 'SecretCode', 'SubmissionLimitPerDay',
       'FeatureE', 'CompEndTime Year', 'CompEndTime Month',
       'CompEndTime Day_of_week', 'CompStartTime Year', 'CompStartTime Month',
       'CompStartTime Day_of_week'],
      dtype='object')

In [60]:
competitions['compDuration'] = ((competitions['CompEndTime Year'] - competitions['CompStartTime Year']) * 12) + \
(competitions['CompEndTime Month'] - competitions['CompStartTime Month']) + 1

In [61]:
competitions.dtypes

CompID                        object
Country                       object
FeatureA                      object
FeatureB                      object
FeatureC                     float64
Kind                           int64
Points Reward                  int64
FeatureD                       int64
SecretCode                     int64
SubmissionLimitPerDay        float64
FeatureE                      object
CompEndTime Year             float64
CompEndTime Month            float64
CompEndTime Day_of_week      float64
CompStartTime Year           float64
CompStartTime Month          float64
CompStartTime Day_of_week      int64
compDuration                 float64
dtype: object

**Adding supplementary information about ongoing Competitions Profile within user activity prediction time period**

In [63]:
def extract_comp_features(df):
    
    year = df['year']
    
    month = df['month']
    
    #extract competitions possibly active at timeline of prediction, must have either started before prediction timeline or
    #will still be active at prediction time, endtime should be year of prediction or later
    
    comp = competitions[(year >= competitions['CompStartTime Year'])]
    
    
    #extract the time difference between prediction time & already started competions
    comp_delta_user = ((year - comp['CompStartTime Year']) * 12) + ((month - comp['CompStartTime Month'])) + 1
    
    
    #select competions happening on that month & year. Not all have end date, so can't use duration alone
    comp_1 = comp[(comp['CompStartTime Year'] == year) & (comp['CompStartTime Month'] == month)]
    
    #select competitions where the comp duration is within prediction timeline
    comp_2 = comp[comp['compDuration'] >= comp_delta_user]
    
    
    comp_3 = comp[comp['CompEndTime Year'] >= year]
    
    comp = pd.concat([comp_1, comp_2, comp_3])
    
    comp = comp[~comp.duplicated(keep = 'last')]
    
    
    #no of Comp features
    noComp = len(comp)
    
    noCompsec = comp['SecretCode'].sum()
    
    noCompnosec = noComp-noCompsec
    
    
    #Comp start statistics
    
    medCompMonth = comp['CompStartTime Month'].median()
    
    medCompDayweek = comp['CompStartTime Day_of_week'].median()
    
    
    #points stats
    minPoints = comp['Points Reward'].min()
    
    maxPoints = comp['Points Reward'].max()
    
    meanPoints = comp['Points Reward'].mean()
    
    medPoints = comp['Points Reward'].median()
    
    sumPoints = comp['Points Reward'].sum()
    
    
    #submission stats
    minSublimit = comp['SubmissionLimitPerDay'].min()
    
    maxSublimit = comp['SubmissionLimitPerDay'].max()
    
    meanSublimit = comp['SubmissionLimitPerDay'].mean()
    
    medSublimit = comp['SubmissionLimitPerDay'].median()
    
    #duration stats
    minDuration = comp['compDuration'].min()
    
    maxDuration = comp['compDuration'].max()
    
    meanDuration = comp['compDuration'].mean()
    
    medDuration = comp['compDuration'].median()
    
    #mode stats
    
    modeFeatA = comp['FeatureA'].mode()[0]
    
    modeFeatB = comp['FeatureB'].mode()[0]
    
    modeFeatE = comp['FeatureE'].mode()[0]
    
    modeCountry = comp['Country'].mode()[0]
    
    
    #FeatureC stats
    
    minFeatC = comp['FeatureC'].min()
    
    maxFeatC = comp['FeatureC'].max()
    
    meanFeatC = comp['FeatureC'].mean()
    
    medFeatC = comp['FeatureC'].median()
    
    sumFeatC = comp['FeatureC'].sum()
    
    
    #FeatureD stats
    
    modeFeatD = comp['FeatureD'].mode()[0]
    
    sumFeatD = comp['FeatureD'].sum()
    
    meanFeatD = comp['FeatureD'].mean()
    
    

    #Kind stats
    sumKind = comp['Kind'].sum()
    
    meanKind = comp['Kind'].mean()
    
    
    
    return pd.Series([noComp, noCompsec, noCompnosec, medCompMonth, medCompDayweek, minPoints, maxPoints,
                      meanPoints, medPoints, minSublimit, maxSublimit, meanSublimit, medSublimit, 
                      minDuration, maxDuration, meanDuration,  medDuration,  modeFeatA,  modeFeatB, 
                      modeFeatE, modeCountry, minFeatC, maxFeatC, meanFeatC, medFeatC, sumFeatC, 
                      modeFeatD, sumFeatD, meanFeatD, sumKind, meanKind,  sumPoints])
    
    

In [64]:
all_data[['noComp', 'noCompsec', 'noCompnosec', 'medCompMonth', 'medCompDayweek', 'minCompPoints', 'maxCompPoints',\
          'meanCompPoints', 'medCompPoints', 'minSublimit', 'maxSublimit', 'meanSublimit', 'medSublimit', 
         'minCompDuration', 'maxCompDuration', 'meanCompDuration',  'medCompDuration', 'modeCompFeatA',  
         'modeCompFeatB', 'modeCompFeatE', 'modeCompCountry', 'minCompFeatC', 'maxCompFeatC', 'meanCompFeatC', 
         'medCompFeatC', 'sumCompFeatC', 'modeCompFeatD', 'sumCompFeatD', \
          'meanCompFeatD', 'sumCompKind', 'meanCompKind', 'sumPoints']] = \
all_data.progress_apply(extract_comp_features, axis = 1)

100%|██████████████████████████████████| 325055/325055 [25:07<00:00, 215.69it/s]


In [65]:
import gc
gc.collect()

0

### 2) Overall user historical user activity behaviour based on training dataframe- Extracting historical behaviour from prior months

**Create feature join month, users who just joined the platform are probably joining due to a competition at that time, so at time of joining, probability of being active that month is higher**

In [66]:
def create_joinmonth(df):
    
    if (df['year'] == df['UserDate Year']) & (df['month'] == df['UserDate Month']):
        
        return 1
    
    else:
        
        return 0

In [67]:
all_data['is_joinmonth'] = all_data.apply(create_joinmonth, axis =1)

In [68]:
all_data[all_data['is_joinmonth'] == 1]['Target'].mean()

0.6346819665644475

In [69]:
all_data[all_data['is_joinmonth'] == 0]['Target'].mean()

0.09796589514802975

In [70]:
import gc
gc.collect()

0

**Adding supplementary information about Historical User Activity**

In [71]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,meanCompFeatC,medCompFeatC,sumCompFeatC,modeCompFeatD,sumCompFeatD,meanCompFeatD,sumCompKind,meanCompKind,sumPoints,is_joinmonth
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,12.539474,9.0,953.0,2,180,2.250000,26,0.325000,4155,0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,13.121212,10.0,433.0,2,71,1.972222,23,0.638889,5173,0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,13.764706,10.0,468.0,2,74,2.000000,24,0.648649,5175,0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,12.539474,9.0,953.0,2,180,2.250000,26,0.325000,4155,0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,12.539474,9.0,953.0,2,180,2.250000,26,0.325000,4155,0


In [72]:
all_data['TargetSum'] = all_data[['CompPart', 'Comment', 'Sub', 'Disc']].sum(axis = 1)

In [229]:
def hist_user_activity(df):
    
    year = df['year']
    
    month = df['month']
    
    user = df['UserID']
    
    train_user = all_data[all_data['UserID'] == user]
    
    
    #select all past years data
    train_user_time_1 = train_user[(train_user['year'] < year)]
    
    #select present year data, prior months
    train_user_time_2 = train_user[(train_user['year'] == year) & (train_user['month'] < month)]
    
    
    train_user_time = pd.concat([train_user_time_1, train_user_time_2])
    
    train_user_time = train_user_time.sort_values(['year', 'month'], ascending = False)
    
    Months_elapsed = len(train_user_time)
    
    
    #Restricting to historical no of months with Target
    train_user_time = train_user_time[train_user_time['Target'].isnull() == False]
    
    HistMonths = len(train_user_time)
    
    HistCompPart = train_user_time['CompPart'].sum()
    
    HistComm = train_user_time['Comment'].sum()
    
    HistSub = train_user_time['Sub'].sum()
    
    HistDisc = train_user_time['Disc'].sum()
    
    HistTotAct = train_user_time['Target'].sum()
    
    HistTargetSum = train_user_time['TargetSum'].sum()
    
    userConsistency = train_user_time['Target'].std()
    
    
    
    try:
        
        HistCompPartRatio = HistCompPart/len(train_user_time)
    
        HistCommRatio = HistComm/len(train_user_time)
    
        HistSubRatio = HistSub/len(train_user_time)
    
        HistDiscRatio = HistDisc/len(train_user_time)
    
        HistTotActRatio = HistTotAct/len(train_user_time)
        
        
    except:
        
        HistCompPartRatio = np.nan
    
        HistCommRatio = np.nan
    
        HistSubRatio = np.nan
    
        HistDiscRatio = np.nan
        
        HistTotActRatio = np.nan
    
    
    #3 months features
    
    extracted = train_user_time.head(3)
    
    Hist3CompPart = extracted['CompPart'].sum()
    
    Hist3Comm = extracted['Comment'].sum()
    
    Hist3Sub = extracted['Sub'].sum()
    
    Hist3Disc = extracted['Disc'].sum()
    
    Hist3TotAct = extracted['Target'].sum()
    
    months_3 = len(extracted)
    
    
    try:
    
        Hist3TotActRatio = extracted['Target'].sum()/len(extracted)
        
    except:
        
        Hist3TotActRatio = np.nan
    
   
    
    
    
    #6 months features

    extracted = train_user_time.head(6)

    Hist6CompPart = extracted['CompPart'].sum()
    
    Hist6Comm = extracted['Comment'].sum()
    
    Hist6Sub = extracted['Sub'].sum()
    
    Hist6Disc = extracted['Disc'].sum()
    
    Hist6TotAct = extracted['Target'].sum()
    
    months_6 = len(extracted)
    
    
    try:
    
        Hist6TotActRatio = extracted['Target'].sum()/len(extracted)
        
    
    except:
        
        Hist6TotActRatio = np.nan
    
    
    
    
    return pd.Series([Months_elapsed, HistMonths, HistCompPart, HistComm, HistSub, HistDisc, HistTotAct, HistTargetSum, userConsistency,
                      HistCompPartRatio, HistCommRatio, HistSubRatio, HistDiscRatio, HistTotActRatio, Hist3CompPart, \
                      Hist3Comm, Hist3Sub, Hist3Disc, Hist3TotAct, Hist3TotActRatio, months_3, Hist6CompPart,  \
                     Hist6Comm, Hist6Sub, Hist6Disc, Hist6TotAct, Hist6TotActRatio, months_6])

In [74]:
#delete the 3 ratios

In [235]:
all_data[['Months_elapsed', 'HistMonths', 'HistCompPart', 'HistComm', 'HistSub', 'HistDisc', 'HistTotAct', 'HistTargetSum', 'userConsistency',\
          'HistCompPartRatio', 'HistCommRatio', 'HistSubRatio', 'HistDiscRatio', 'HistTotActRatio', 'Hist3CompPart', \
          'Hist3Comm', 'Hist3Sub', 'Hist3Disc', 'Hist3TotAct', 'Hist3TotActRatio', 'months_3', 'Hist6CompPart',  \
          'Hist6Comm', 'Hist6Sub', 'Hist6Disc', 'Hist6TotAct', 'Hist6TotActRatio', 'months_6']] = \
all_data.progress_apply(hist_user_activity, axis = 1)

In [147]:
all_data.isnull().sum()

UserID                       0
month                        0
year                         0
CompPart                 65223
Comment                  65223
                         ...  
MeanDiscFeatFPrevMon    321279
MinDiscPrevMon          321279
MeanDiscPrevMon         321279
MedianDiscPrevMon       321279
MaxDiscPrevMon          321279
Length: 205, dtype: int64

In [148]:
import gc
gc.collect()

0

In [ ]:
#lag - 1.38 hrs

In [149]:
#Set to null where, ratio is 0

all_data.loc[all_data['HistCommRatio'].isnull() == True, 'HistComm'] = np.nan
all_data.loc[all_data['HistCompPartRatio'].isnull() == True, 'HistCompPart'] = np.nan
all_data.loc[all_data['HistSubRatio'].isnull() == True, 'HistSub'] = np.nan
all_data.loc[all_data['HistDiscRatio'].isnull() == True, 'HistDisc'] = np.nan
all_data.loc[all_data['HistTotActRatio'].isnull() == True, 'HistTotAct'] = np.nan
all_data.loc[all_data['HistTotActRatio'].isnull() == True, 'userConsistency'] = np.nan
all_data.loc[all_data['HistTotActRatio'].isnull() == True, 'HistTargetSum'] = np.nan


all_data.loc[all_data['Hist3TotActRatio'].isnull() == True, 'Hist3Comm'] = np.nan
all_data.loc[all_data['Hist3TotActRatio'].isnull() == True, 'Hist3CompPart'] = np.nan
all_data.loc[all_data['Hist3TotActRatio'].isnull() == True, 'Hist3Sub'] = np.nan
all_data.loc[all_data['Hist3TotActRatio'].isnull() == True, 'Hist3Disc'] = np.nan
all_data.loc[all_data['Hist3TotActRatio'].isnull() == True, 'Hist3TotAct'] = np.nan



all_data.loc[all_data['Hist6TotActRatio'].isnull() == True, 'Hist6Comm'] = np.nan
all_data.loc[all_data['Hist6TotActRatio'].isnull() == True, 'Hist6CompPart'] = np.nan
all_data.loc[all_data['Hist6TotActRatio'].isnull() == True, 'Hist6Sub'] = np.nan
all_data.loc[all_data['Hist6TotActRatio'].isnull() == True, 'Hist6Disc'] = np.nan
all_data.loc[all_data['Hist6TotActRatio'].isnull() == True, 'Hist6TotAct'] = np.nan

In [150]:
all_data.isnull().sum()

UserID                       0
month                        0
year                         0
CompPart                 65223
Comment                  65223
                         ...  
MeanDiscFeatFPrevMon    321279
MinDiscPrevMon          321279
MeanDiscPrevMon         321279
MedianDiscPrevMon       321279
MaxDiscPrevMon          321279
Length: 205, dtype: int64

**Create additional feature engineering for users with zero standard deviation and additional weighting of overall user behaviour (Historical timelines)**

In [231]:
def create_consistency_index(df, consis, totratio, months):
    
    if df[consis] == 0:
        
        if df[totratio] == 1:
            
            return df[months]
        
        else:
            
            return - df[months]
        
    else:
        
        return 0

In [232]:
def create_consistency_weighting(df, consis, totratio, months, thres = 0.6):
    
    weight = 1- df[consis]
        
    if df[totratio] >= thres:
            
        return round(df[months] * weight, 1)
        
    elif df[totratio] < thres:
        
        return - round(df[months] * weight, 1)
        
    else:
        
        return np.nan

In [233]:
all_data['consistencyIndex'] = all_data.progress_apply(create_consistency_index, axis = 1, \
                                                       args = ['userConsistency', 'HistTotActRatio', \
                                                               'HistMonths'])

100%|████████████████████████████████| 325055/325055 [00:14<00:00, 22220.37it/s]


In [234]:
all_data['consistencyWeighting'] = all_data.progress_apply(create_consistency_weighting, axis = 1, \
                                                          args = ['userConsistency', 'HistTotActRatio', \
                                                               'HistMonths'])

100%|████████████████████████████████| 325055/325055 [00:15<00:00, 20685.54it/s]


In [155]:
all_data['consistencyWeighting_0.8'] = all_data.progress_apply(create_consistency_weighting, axis = 1, \
                                                               args = ['userConsistency', 'HistTotActRatio', \
                                                               'HistMonths', 0.8])

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 29058.54it/s]


In [85]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,Hist6CompPart,Hist6Comm,Hist6Sub,Hist6Disc,Hist6TotAct,Hist6TotActRatio,months_6,consistencyIndex,consistencyWeighting,consistencyWeighting_0.8
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-11.1,-11.1
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,1.0,0.0,1.0,0.0,1.0,0.166667,6.0,0.0,-3.6,-3.6
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-4.4,-4.4
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-11.9,-11.9
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-12.7,-12.7


**Adding supplementary information about Historical User Activity lag features for the last time window**

In [86]:
import gc
gc.collect()

0

In [156]:
def Extract_user_lags(df, window):
    
    timestamp = df['timestamp'] - pd.DateOffset(months = 1)
    
    user = df['UserID']
    
    train_user = all_data[all_data['UserID'] == user]
    
    
    train_user_time = train_user[(train_user['timestamp'] <= timestamp)]
    
    train_user_time = train_user_time.sort_values(['year', 'month'], ascending = False)
    
    
    
    extracted = list(train_user_time['Target'][:window].values)
    
    pad_len = window-len(extracted)
    
    if len(extracted) < window:
        
        extracted = extracted + (pad_len * [np.nan])
        
        
    extracted_sum = list(train_user_time['TargetSum'][:window].values)
    
    pad_len = window-len(extracted_sum)
    
    if len(extracted_sum) < window:
        
        extracted_sum = extracted_sum + (pad_len * [np.nan])
        
        
    
    
    HistCompPartPrev6Mon = train_user_time['CompPart'].head(6).values
    
    HistCommPrev6Mon = train_user_time['Comment'].head(6).values
    
    HistSubPrev6Mon = train_user_time['Sub'].head(6).values
    
    HistDiscPrev6Mon = train_user_time['Disc'].head(6).values
    
    
    extracted.extend(extracted_sum)
    
    extracted.extend([HistCompPartPrev6Mon, HistCommPrev6Mon, HistSubPrev6Mon, HistDiscPrev6Mon])
    
    
    
    
    return pd.Series(extracted) 

**Preprocess in timestamps format**

In [90]:
#Replace the years with real years format to use previous month datetime

year_dict = {1 : 2010, 2 : 2011, 3 : 2012, 4 : 2013}
all_data['year'] = all_data['year'].map(year_dict)
compPart['CompPartCreated Year'] = compPart['CompPartCreated Year'].map(year_dict)
submissions['SubDate Year'] = submissions['SubDate Year'].map(year_dict)
discussions['DiscDate Year'] = discussions['DiscDate Year'].map(year_dict)
comments['CommentDate Year'] = comments['CommentDate Year'].map(year_dict)

In [91]:
compPart['timestamp'] = pd.to_datetime(compPart['CompPartCreated Year'].astype(str) + '-' + compPart['CompPartCreated Month'].astype(str))

submissions['timestamp'] = pd.to_datetime(submissions['SubDate Year'].astype(str) + '-' + submissions['SubDate Month'].astype(str))

discussions['timestamp'] = pd.to_datetime(discussions['DiscDate Year'].astype(str) + '-' + discussions['DiscDate Month'].astype(str))

comments['timestamp'] = pd.to_datetime(comments['CommentDate Year'].astype(str) + '-' + comments['CommentDate Month'].astype(str))

all_data['timestamp'] = pd.to_datetime(all_data['year'].astype(str) + '-' + all_data['month'].astype(str))

In [92]:
from datetime import timedelta
compPart[compPart['timestamp'] == compPart['timestamp'].iloc[0] - pd.DateOffset(months= 1)]

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week,timestamp
11,ID_FBDX,ID_1N5J2PGO,NaN,NaN,2011,12,6,2011-12-01
21,ID_HJ9S,ID_1N5J2PGO,9.0,6.0,2011,12,6,2011-12-01
28,ID_XWMG,ID_1N5J2PGO,NaN,NaN,2011,12,6,2011-12-01
32,ID_MPSN,ID_Y3FUR285,NaN,NaN,2011,12,7,2011-12-01
34,ID_1INW,ID_Y3FUR285,NaN,NaN,2011,12,7,2011-12-01
...,...,...,...,...,...,...,...,...
47849,ID_XWMG,ID_5KPFLEJ8,NaN,NaN,2011,12,7,2011-12-01
47856,ID_XWMG,ID_AJHFNSLQ,NaN,NaN,2011,12,6,2011-12-01
47860,ID_XWMG,ID_4B1GPG9O,NaN,NaN,2011,12,7,2011-12-01
47876,ID_XWMG,ID_5M2CCZFE,11.0,3.0,2011,12,4,2011-12-01


In [93]:
year_dict = {2010 : 1, 2011 : 2, 2012 : 3,  2013 : 4}
all_data['year'] = all_data['year'].map(year_dict)
compPart['CompPartCreated Year'] = compPart['CompPartCreated Year'].map(year_dict)
submissions['SubDate Year'] = submissions['SubDate Year'].map(year_dict)
discussions['DiscDate Year'] = discussions['DiscDate Year'].map(year_dict)
comments['CommentDate Year'] = comments['CommentDate Year'].map(year_dict)

In [94]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,Hist6Comm,Hist6Sub,Hist6Disc,Hist6TotAct,Hist6TotActRatio,months_6,consistencyIndex,consistencyWeighting,consistencyWeighting_0.8,timestamp
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-11.1,-11.1,2012-08-01
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,1.0,0.0,1.0,0.166667,6.0,0.0,-3.6,-3.6,2011-08-01
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-4.4,-4.4,2011-09-01
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-11.9,-11.9,2012-09-01
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,-12.7,-12.7,2012-10-01


In [157]:
#Use lag of 6 months for user behaviour to predict new months

all_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_1_TargetSum', 'lag_2_TargetSum', \
          'lag_3_TargetSum', 'lag_4_TargetSum', 'lag_5_TargetSum', 'lag_6_TargetSum', \
          'HistCompPartPrev6Mon', 'HistCommPrev6Mon', \
          'HistSubPrev6Mon', 'HistDiscPrev6Mon']] = \
all_data.progress_apply(Extract_user_lags, axis = 1, args = [6])

100%|█████████████████████████████████| 325055/325055 [1:36:47<00:00, 55.98it/s]


**Preprocess**

In [158]:
def temp(df, col, ax):
    
    try:
        
        return df[col][ax]
    
    
    except:
        
        return np.nan

In [159]:
all_data['HistCompPartPrevMon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 0], axis = 1)

all_data['HistCommPrevMon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 0], axis = 1)

all_data['HistDiscPrevMon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 0], axis = 1)

all_data['HistSubPrevMon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 0], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28168.58it/s]


In [160]:
all_data['HistCompPartPrev2Mon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 1], axis = 1)

all_data['HistCommPrev2Mon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 1], axis = 1)

all_data['HistDiscPrev2Mon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 1], axis = 1)

all_data['HistSubPrev2Mon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 1], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:12<00:00, 26989.87it/s]


In [161]:
all_data['HistCompPartPrev3Mon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 2], axis = 1)

all_data['HistCommPrev3Mon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 2], axis = 1)

all_data['HistDiscPrev3Mon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 2], axis = 1)

all_data['HistSubPrev3Mon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 2], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28073.62it/s]


In [162]:
all_data['HistCompPartPrev4Mon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 3], axis = 1)

all_data['HistCommPrev4Mon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 3], axis = 1)

all_data['HistDiscPrev4Mon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 3], axis = 1)

all_data['HistSubPrev4Mon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 3], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28159.68it/s]


In [163]:
all_data['HistCompPartPrev5Mon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 4], axis = 1)

all_data['HistCommPrev5Mon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 4], axis = 1)

all_data['HistDiscPrev5Mon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 4], axis = 1)

all_data['HistSubPrev5Mon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 4], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28039.09it/s]


In [164]:
all_data['HistCompPartPrev6Mon'] = all_data.progress_apply(temp, args = ['HistCompPartPrev6Mon', 5], axis = 1)

all_data['HistCommPrev6Mon'] = all_data.progress_apply(temp, args = ['HistCommPrev6Mon', 5], axis = 1)

all_data['HistDiscPrev6Mon'] = all_data.progress_apply(temp, args = ['HistDiscPrev6Mon', 5], axis = 1)

all_data['HistSubPrev6Mon'] = all_data.progress_apply(temp, args = ['HistSubPrev6Mon', 5], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:12<00:00, 25835.12it/s]


**Create additional feature engineering for users with zero standard deviation and additional weighting of user behaviour(3 & 6 Months).**

In [165]:
all_data['userConsistency6Mon'] = all_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6']].std(axis = 1)

all_data['userConsistency3Mon'] = all_data[['lag_1', 'lag_2', 'lag_3']].std(axis = 1)


In [166]:
all_data['6monconsistencyIndex'] = all_data.progress_apply(create_consistency_index, \
                                                           args =['userConsistency6Mon', 'Hist6TotActRatio', \
                                                               'months_6'], axis = 1)

all_data['3monconsistencyIndex'] = all_data.progress_apply(create_consistency_index, \
                                                           args =['userConsistency3Mon', 'Hist3TotActRatio', \
                                                               'months_3'], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:10<00:00, 31088.29it/s]


In [167]:
all_data['6monconsistencyWeighting'] = all_data.progress_apply(create_consistency_weighting, \
                                                               args =['userConsistency6Mon', 'Hist6TotActRatio', \
                                                               'months_6'], axis = 1)

all_data['3monconsistencyWeighting'] = all_data.progress_apply(create_consistency_weighting, \
                                                               args =['userConsistency3Mon', 'Hist3TotActRatio', \
                                                               'months_3'], axis = 1)

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28142.43it/s]


In [168]:
all_data['6monconsistencyWeighting_0.8'] = all_data.progress_apply(create_consistency_weighting, axis = 1, \
                                                                   args = ['userConsistency6Mon', 'Hist6TotActRatio', \
                                                               'months_6', 0.8])

all_data['3monconsistencyWeighting_0.8'] = all_data.progress_apply(create_consistency_weighting, axis = 1, \
                                                                   args = ['userConsistency3Mon', 'Hist3TotActRatio', \
                                                               'months_3', 0.8])

100%|████████████████████████████████| 325055/325055 [00:11<00:00, 28534.57it/s]


In [169]:
import gc
gc.collect()

0

In [108]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,HistDiscPrev5Mon,HistSubPrev5Mon,userConsistency6Mon,userConsistency3Mon,6monconsistencyIndex,3monconsistencyIndex,6monconsistencyWeighting,3monconsistencyWeighting,6monconsistencyWeighting_0.8,3monconsistencyWeighting_0.8
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.000000,0.0,-6.0,-3.0,-6.0,-3.0,-6.0,-3.0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,1.0,0.447214,0.0,0.0,-3.0,-3.3,-3.0,-3.3,-3.0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.408248,0.0,0.0,-3.0,-3.6,-3.0,-3.6,-3.0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.000000,0.0,-6.0,-3.0,-6.0,-3.0,-6.0,-3.0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,0.000000,0.0,-6.0,-3.0,-6.0,-3.0,-6.0,-3.0


In [109]:
import gc
gc.collect()

0

**Creating aggregate feature on overall user behaviour for stronger signal**

In [170]:
all_data['OverallRatiouserbev'] = all_data['HistCompPartRatio'] + all_data['HistCommRatio'] + all_data['HistSubRatio'] + \
all_data['HistDiscRatio']


all_data['Overalluserbev'] = all_data['HistCompPart'] + all_data['HistComm'] + all_data['HistSub'] + \
all_data['HistDisc']


all_data['Overall3userbev'] = all_data['Hist3CompPart'] + all_data['Hist3Comm'] + all_data['Hist3Sub'] + \
all_data['Hist3Disc']

all_data['Overall6userbev'] = all_data['Hist6CompPart'] + all_data['Hist6Comm'] + all_data['Hist6Sub'] + \
all_data['Hist6Disc']

In [171]:
all_data.groupby(['OverallRatiouserbev'])['Target'].mean().tail(10)

OverallRatiouserbev
3.500000    0.754717
3.600000    1.000000
3.666667    0.666667
3.750000    0.666667
3.777778    1.000000
3.800000    1.000000
3.833333    0.500000
3.857143    1.000000
3.875000    1.000000
4.000000    0.597701
Name: Target, dtype: float64

In [172]:
all_data.groupby(['Overalluserbev'])['Target'].mean().tail(10)

Overalluserbev
78.0    1.000000
79.0    1.000000
81.0    1.000000
82.0    1.000000
84.0         NaN
85.0    0.666667
87.0    1.000000
88.0    0.000000
91.0    1.000000
95.0         NaN
Name: Target, dtype: float64

In [173]:
all_data.groupby(['Overall3userbev'])['Target'].mean().tail(10)

Overall3userbev
3.0     0.295945
4.0     0.431966
5.0     0.513367
6.0     0.588530
7.0     0.690361
8.0     0.726477
9.0     0.817109
10.0    0.893401
11.0    0.917910
12.0    0.928571
Name: Target, dtype: float64

In [174]:
all_data.groupby(['Overall6userbev'])['Target'].mean().tail(10)

Overall6userbev
15.0    0.752941
16.0    0.809524
17.0    0.875000
18.0    0.911392
19.0    0.863014
20.0    0.897959
21.0    0.944444
22.0    0.965517
23.0    0.961538
24.0    0.947368
Name: Target, dtype: float64

### 3) Overall user historical user activity statistics based on supplementary datasets- Extracting historical behaviour from prior months

**Adding supplementary information about users historical competition participation rankings and submission counts**

In [115]:
comp_merge = competitions

In [116]:
compPart = compPart.merge(comp_merge, on = ['CompID'], how = 'left')

In [117]:
compPart.columns

Index(['CompID', 'UserID', 'PublicRank', 'Successful Submission Count',
       'CompPartCreated Year', 'CompPartCreated Month',
       'CompPartCreated Day_of_week', 'timestamp', 'Country', 'FeatureA',
       'FeatureB', 'FeatureC', 'Kind', 'Points Reward', 'FeatureD',
       'SecretCode', 'SubmissionLimitPerDay', 'FeatureE', 'CompEndTime Year',
       'CompEndTime Month', 'CompEndTime Day_of_week', 'CompStartTime Year',
       'CompStartTime Month', 'CompStartTime Day_of_week', 'compDuration'],
      dtype='object')

In [118]:
compPart

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week,timestamp,Country,FeatureA,...,SecretCode,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week,compDuration
0,ID_WMUF,ID_UWBBZ9OF,11.0,10.0,3,1,4,2012-01-01,ID_CKI7,"[3, 2]",...,0,30.0,[2],3.0,1.0,6.0,3.0,1.0,4,1.0
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7,2012-01-01,NaN,[1],...,0,10.0,"[1, 5]",NaN,NaN,NaN,2.0,2.0,7,NaN
2,ID_WMUF,ID_1N5J2PGO,NaN,10.0,3,1,5,2012-01-01,ID_CKI7,"[3, 2]",...,0,30.0,[2],3.0,1.0,6.0,3.0,1.0,4,1.0
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5,2011-10-01,ID_I1L9,[1],...,1,200.0,[1],2.0,10.0,5.0,2.0,10.0,5,1.0
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4,2011-05-01,ID_GU74,[1],...,0,30.0,[1],2.0,9.0,7.0,2.0,5.0,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48560,ID_XYJZ,ID_5C4D0VO2,11.0,10.0,3,7,6,2012-07-01,ID_CKI7,[1],...,1,50.0,[1],3.0,7.0,7.0,3.0,7.0,5,1.0
48561,ID_XYJZ,ID_JRJZQB8S,11.0,10.0,3,7,5,2012-07-01,ID_CKI7,[1],...,1,50.0,[1],3.0,7.0,7.0,3.0,7.0,5,1.0
48562,ID_XYJZ,ID_H4FM7RBV,NaN,NaN,3,7,5,2012-07-01,ID_CKI7,[1],...,1,50.0,[1],3.0,7.0,7.0,3.0,7.0,5,1.0
48563,ID_XYJZ,ID_C3QOEMU4,11.0,10.0,3,7,5,2012-07-01,ID_CKI7,[1],...,1,50.0,[1],3.0,7.0,7.0,3.0,7.0,5,1.0


In [119]:
compPart['compuser_start'] = ((compPart['CompPartCreated Year'] - compPart['CompStartTime Year']) * 12) + \
(compPart['CompPartCreated Month'] - compPart['CompStartTime Month'])

In [120]:
compPart.head()

,CompID,UserID,PublicRank,Successful Submission Count,CompPartCreated Year,CompPartCreated Month,CompPartCreated Day_of_week,timestamp,Country,FeatureA,...,SubmissionLimitPerDay,FeatureE,CompEndTime Year,CompEndTime Month,CompEndTime Day_of_week,CompStartTime Year,CompStartTime Month,CompStartTime Day_of_week,compDuration,compuser_start
0,ID_WMUF,ID_UWBBZ9OF,11.0,10.0,3,1,4,2012-01-01,ID_CKI7,"[3, 2]",...,30.0,[2],3.0,1.0,6.0,3.0,1.0,4,1.0,0.0
1,ID_MPSN,ID_UWBBZ9OF,NaN,NaN,3,1,7,2012-01-01,NaN,[1],...,10.0,"[1, 5]",NaN,NaN,NaN,2.0,2.0,7,NaN,11.0
2,ID_WMUF,ID_1N5J2PGO,NaN,10.0,3,1,5,2012-01-01,ID_CKI7,"[3, 2]",...,30.0,[2],3.0,1.0,6.0,3.0,1.0,4,1.0,0.0
3,ID_7MLO,ID_1N5J2PGO,NaN,NaN,2,10,5,2011-10-01,ID_I1L9,[1],...,200.0,[1],2.0,10.0,5.0,2.0,10.0,5,1.0,0.0
4,ID_2KEY,ID_1N5J2PGO,NaN,NaN,2,5,4,2011-05-01,ID_GU74,[1],...,30.0,[1],2.0,9.0,7.0,2.0,5.0,4,5.0,0.0


In [121]:
def extract_comppart_stats(df):
    
    year = df['year']
    
    month = df['month']
    
    user = df['UserID']
    
    comp_user = compPart[compPart['UserID'] == user]
    
    
    #select all past years data
    comp_user_time_1 = comp_user[(comp_user['CompPartCreated Year'] < year)]
    
    #select present year data, prior months
    comp_user_time_2 = comp_user[(comp_user['CompPartCreated Year'] == year) & (comp_user['CompPartCreated Month'] < month)]
    
    
    
    comp_user_time = pd.concat([comp_user_time_1, comp_user_time_2])
    
    comp_user_time_group = comp_user_time.groupby(['UserID', 'CompPartCreated Year', 'CompPartCreated Month'])\
    ['CompID'].count()
    
    

    noCompPartall = len(comp_user_time)
    
    MinCompPartall = comp_user_time_group.min()
    
    MeanCompPartall = comp_user_time_group.mean()
    
    MedianCompPartall = comp_user_time_group.median()
    
    MaxCompPartall = comp_user_time_group.max()
    
    
    
    minpublicRank = comp_user_time['PublicRank'].min()
    
    maxpublicRank = comp_user_time['PublicRank'].max()
    
    medpublicRank = comp_user_time['PublicRank'].median()
    
    meanpublicRank = comp_user_time['PublicRank'].mean()
    

    
    medsubCount = comp_user_time['Successful Submission Count'].median()
    
    minsubCount = comp_user_time['Successful Submission Count'].min()
    
    maxsubCount = comp_user_time['Successful Submission Count'].max()
    
    meansubCount = comp_user_time['Successful Submission Count'].mean()
    
    SumSubCount = comp_user_time['Successful Submission Count'].sum()
    
    
    
    meduserstart = comp_user_time['compuser_start'].median()
    
    minuserstart = comp_user_time['compuser_start'].min()
    
    maxuserstart = comp_user_time['compuser_start'].max()
    
    meanuserstart = comp_user_time['compuser_start'].mean()
    
    
    
    return pd.Series([noCompPartall, minpublicRank, maxpublicRank, medpublicRank, meanpublicRank, \
                      SumSubCount, medsubCount, minsubCount, maxsubCount, meansubCount, meduserstart, \
                      minuserstart, maxuserstart, meanuserstart,  MinCompPartall, MeanCompPartall, \
                      MedianCompPartall, MaxCompPartall])

In [122]:
all_data[['noCompPartall', 'minpublicRank', 'maxpublicRank', 'medpublicRank', 'meanpublicRank',  'SumSubCount',
          'medsubCount', 'minsubCount', 'maxsubCount', 'meansubCount', 'meduserstart', \
          'minuserstart', 'maxuserstart', 'meanuserstart',  'MinCompPartall', \
         'MeanCompPartall', 'MedianCompPartall', 'MaxCompPartall' ]] = all_data.progress_apply(extract_comppart_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [27:46<00:00, 195.06it/s]


In [123]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,maxsubCount,meansubCount,meduserstart,minuserstart,maxuserstart,meanuserstart,MinCompPartall,MeanCompPartall,MedianCompPartall,MaxCompPartall
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,10.0,10.0,3.0,2.0,4.0,3.0,1.0,1.0,1.0,1.0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,10.0,10.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,10.0,10.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,10.0,10.0,3.0,2.0,4.0,3.0,1.0,1.0,1.0,1.0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,10.0,10.0,3.0,2.0,4.0,3.0,1.0,1.0,1.0,1.0


**Adding supplementary information about users historical total discussions**

In [124]:
def extract_disc_stats(df):
    
    year = df['year']
    
    month = df['month']
    
    user = df['UserID']
    
    disc_user = discussions[discussions['UserID'] == user]
    
    
    #select all past years data
    disc_user_time_1 = disc_user[(disc_user['DiscDate Year'] < year)]
    
    #select present year data, prior months
    disc_user_time_2 = disc_user[(disc_user['DiscDate Year'] == year) & (disc_user['DiscDate Month'] < month)]
    
    
    
    disc_user_time = pd.concat([disc_user_time_1, disc_user_time_2])
    
    disc_user_time_group = disc_user_time.groupby(['UserID', 'DiscDate Year', 'DiscDate Month'])['DiscID'].count()
    
    
    noDiscStatsall = len(disc_user_time)
    
    
    SumDiscFeatFall = disc_user_time['FeatureF'].sum()
    
    MeanDiscFeatFall = disc_user_time['FeatureF'].mean()
    
    
    MinDiscall = disc_user_time_group.min()
    
    MeanDiscall = disc_user_time_group.mean()
    
    MedianDiscall = disc_user_time_group.median()
    
    MaxDiscall = disc_user_time_group.max()
    
    

    
    return pd.Series([noDiscStatsall, SumDiscFeatFall,  MeanDiscFeatFall, MinDiscall, MeanDiscall,  \
                      MedianDiscall,  MaxDiscall])

In [125]:
all_data[['noDiscStatsall', 'SumDiscFeatFall',  'MeanDiscFeatFall', 'MinDiscall', 'MeanDiscall',  \
          'MedianDiscall',  'MaxDiscall']] = all_data.progress_apply(extract_disc_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [11:09<00:00, 485.75it/s]


**Adding supplementary information about users historical total submissions**

In [126]:
def extract_sub_stats(df):
    
    year = df['year']
    
    month = df['month']
    
    user = df['UserID']
    
    sub_user = submissions[submissions['UserID'] == user]
    
    
    #select all past years data
    sub_user_time_1 = sub_user[(sub_user['SubDate Year'] < year)]
    
    #select present year data, prior months
    sub_user_time_2 = sub_user[(sub_user['SubDate Year'] == year) & (sub_user['SubDate Month'] < month)]
    
    
    
    sub_user_time = pd.concat([sub_user_time_1, sub_user_time_2])
    
    sub_user_time_group = sub_user_time.groupby(['UserID', 'SubDate Year', 'SubDate Month'])['UserID'].count()
    
    
    noSubStatsall = len(sub_user_time)
    
    
    SumSubFeatGall = sub_user_time['FeatureG'].sum()
    
    MeanSubFeatGall = sub_user_time['FeatureG'].mean()
    
    
    
    MinSuball = sub_user_time_group.min()
    
    MeanSuball = sub_user_time_group.mean()
    
    MedianSuball = sub_user_time_group.median()
    
    MaxSuball = sub_user_time_group.max()
    
    

    
    return pd.Series([noSubStatsall, SumSubFeatGall,  MeanSubFeatGall, MinSuball, MeanSuball,  \
                      MedianSuball,  MaxSuball])

In [127]:
all_data[['noSubStatsall', 'SumSubFeatGall', 'MeanSubFeatGall', 'MinSuball', 'MeanSuball',  'MedianSuball', \
          'MaxSuball']] = all_data.progress_apply(extract_sub_stats, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:52:38<00:00, 48.10it/s]


**Adding supplementary information about users historical total comments**

In [128]:
def extract_comm_stats(df):
    
    year = df['year']
    
    month = df['month']
    
    user = df['UserID']
    
    comm_user = comments[comments['UserID'] == user]
    
    
    #select all past years data
    comm_user_time_1 = comm_user[(comm_user['CommentDate Year'] < year)]
    
    #select present year data, prior months
    comm_user_time_2 = comm_user[(comm_user['CommentDate Year'] == year) & (comm_user['CommentDate Month'] < month)]
    
    
    
    comm_user_time = pd.concat([comm_user_time_1, comm_user_time_2])
    
    
    
    noCommStatsall = len(comm_user_time)
    
    

    
    return noCommStatsall

In [129]:
all_data['noCommStatsall'] = all_data.progress_apply(extract_comm_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [09:02<00:00, 598.95it/s]


In [130]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,MedianDiscall,MaxDiscall,noSubStatsall,SumSubFeatGall,MeanSubFeatGall,MinSuball,MeanSuball,MedianSuball,MaxSuball,noCommStatsall
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,5.0,3.0,0.6,2.0,2.5,2.5,3.0,0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,2.0,1.0,0.5,2.0,2.0,2.0,2.0,0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,2.0,1.0,0.5,2.0,2.0,2.0,2.0,0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,5.0,3.0,0.6,2.0,2.5,2.5,3.0,0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,5.0,3.0,0.6,2.0,2.5,2.5,3.0,0


**Determine total number of competitions that have been available to the user so far, since user start date**

In [131]:
def extract_totalcomp_stats(df):
    
    
    user = df['UserID']
    
    year = df['year']
    
    month = df['month']
    
    
    year_user =  users[users['UserID'] == user]['UserDate Year'].values[0]
    
    month_user = users[users['UserID'] == user]['UserDate Month'].values[0]
    
    
    
    #select competitions that started before user joined
    
    comp_time_1 = competitions[competitions['CompStartTime Year'] < year_user]
    

    # extract the time difference between user join time & already started competions, ensure user will meet this competition
    # when he joins the platform
    comp_delta_user = ((year_user - comp_time_1['CompStartTime Year']) * 12) + \
    (month_user - comp_time_1['CompStartTime Month']) + 1
    
    # select competitions where the comp duration is within user timeline
    comp_1 = comp_time_1[comp_time_1['compDuration'] >= comp_delta_user]
    
    
    
    #select competitions that started after or when user joined
    
    comp_time_2 = competitions[competitions['CompStartTime Year'] >= year_user]
    
    
    #select competitions that would have occurred up till prediction time
    
    comp_2 = comp_time_2[competitions['CompStartTime Year'] < year]
    
    
    #select present year data, prior months
    comp_3= comp_time_2[(comp_time_2['CompStartTime Year'] == year) & (comp_time_2['CompStartTime Month'] < month)]
    
    
    comp_time = pd.concat([comp_1, comp_2, comp_3])
    
    comp_time = comp_time[~comp_time.duplicated(keep = 'last')]
    
    
    
    TotalNoComp = len(comp_time)
    
    

    
    return TotalNoComp

In [132]:
all_data['TotalNoComp'] = all_data.progress_apply(extract_totalcomp_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [34:39<00:00, 156.32it/s]


**Additional features on the ratio of disccussions, submissions, comments, competitions interacted with to no of competitions available so far (user frequency of competitions interactions)**

In [133]:
import gc
gc.collect()

0

In [134]:
all_data['total_disc_compratio'] = all_data['noDiscStatsall']/all_data['TotalNoComp']

all_data['total_sub_compratio'] = all_data['noSubStatsall']/all_data['TotalNoComp']

all_data['total_comm_compratio'] = all_data['noCommStatsall']/all_data['TotalNoComp']

all_data.loc[all_data['HistCompPart'].isnull() == True, 'noCompPartall'] = np.nan

all_data['totalcompInteractionratio'] = all_data['noCompPartall']/all_data['TotalNoComp']


### 4) The current user behaviour from the previous month is an indicator of next month behaviour, adding previous month feature statistics about supplementary datasets

In [135]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,MinSuball,MeanSuball,MedianSuball,MaxSuball,noCommStatsall,TotalNoComp,total_disc_compratio,total_sub_compratio,total_comm_compratio,totalcompInteractionratio
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,2.5,2.5,3.0,0,104,0.0,0.048077,0.0,0.019231
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,2.0,2.0,2.0,0,24,0.0,0.083333,0.0,0.041667
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,2.0,2.0,2.0,0,26,0.0,0.076923,0.0,0.038462
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,2.5,2.5,3.0,0,106,0.0,0.047170,0.0,0.018868
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,2.5,2.5,3.0,0,109,0.0,0.045872,0.0,0.018349


**Adding supplementary information about user current participation from previous month**

**Create feature on most recent competition participation from the previous month**

In [136]:
def extract_compprevmonth_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    
    #select previous month data
    
    comp_user_time = comp_user[comp_user['timestamp'] == prev_month]
    
    #points stats
    sumPointsPrevMon = comp_user_time['Points Reward'].sum()
    
    
    
    #no of competitions
    
    noCompPartPrevMon = len(comp_user_time)
    
    noCompPartPrevMonsec = comp_user_time['SecretCode'].sum()
    
    noCompPartPrevMonnosec = noCompPartPrevMon - noCompPartPrevMonsec
    
    
    
    minRankPrevMon = comp_user_time['PublicRank'].min()
    
    maxRankPrevMon = comp_user_time['PublicRank'].max()
    
    medRankPrevMon = comp_user_time['PublicRank'].median()
    
    meanRankPrevMon= comp_user_time['PublicRank'].mean()
    

    
    medsubCountPrevMon = comp_user_time['Successful Submission Count'].median()
    
    minsubCountPrevMon = comp_user_time['Successful Submission Count'].min()
    
    maxsubCountPrevMon = comp_user_time['Successful Submission Count'].max()
    
    meansubCountPrevMon = comp_user_time['Successful Submission Count'].mean()
    
    SumSubCountPrevMon = comp_user_time['Successful Submission Count'].sum()
    
    
    #points stats
    minPointsPrevMon = comp_user_time['Points Reward'].min()
    
    maxPointsPrevMon = comp_user_time['Points Reward'].max()
    
    meanPointsPrevMon = comp_user_time['Points Reward'].mean()
    
    medPointsPrevMon = comp_user_time['Points Reward'].median()
    
    
    #duration stats
    minDurationPrevMon = comp_user_time['compDuration'].min()
    
    maxDurationPrevMon = comp_user_time['compDuration'].max()
    
    meanDurationPrevMon = comp_user_time['compDuration'].mean()
    
    medDurationPrevMon = comp_user_time['compDuration'].median()
    
    
    #Kind
    
    SumKindPrevMon = comp_user_time['Kind'].sum()
    
    
    meduserstartPrevMon = comp_user_time['compuser_start'].median()
    
    minuserstartPrevMon = comp_user_time['compuser_start'].min()
    
    maxuserstartPrevMon = comp_user_time['compuser_start'].max()
    
    meanuserstartPrevMon = comp_user_time['compuser_start'].mean()
    
    
    
    
    
    return pd.Series([noCompPartPrevMon, noCompPartPrevMonsec,  noCompPartPrevMonnosec, minRankPrevMon, maxRankPrevMon, medRankPrevMon,  meanRankPrevMon, \
                      medsubCountPrevMon, minsubCountPrevMon,  maxsubCountPrevMon, meansubCountPrevMon, \
                     SumSubCountPrevMon, minPointsPrevMon, maxPointsPrevMon, meanPointsPrevMon, medPointsPrevMon, \
                     minDurationPrevMon , maxDurationPrevMon,  meanDurationPrevMon,  medDurationPrevMon, SumKindPrevMon, \
                     sumPointsPrevMon,  meduserstartPrevMon, minuserstartPrevMon, maxuserstartPrevMon, meanuserstartPrevMon])

In [137]:
import gc
gc.collect()

0

In [138]:
all_data[['noCompPartPrevMon', 'noCompPartPrevMonsec', 'noCompPartPrevMonnosec', 'minRankPrevMon', 'maxRankPrevMon', 'medRankPrevMon',  'meanRankPrevMon', \
          'medsubCountPrevMon', 'minsubCountPrevMon',  'maxsubCountPrevMon', 'meansubCountPrevMon', \
          'SumSubCountPrevMon', 'minPointsPrevMon', 'maxPointsPrevMon', 'meanPointsPrevMon', \
          'medPointsPrevMon', 'minDurationPrevMon' , 'maxDurationPrevMon',  'meanDurationPrevMon',  \
          'medDurationPrevMon', 'SumKindPrevMon', 'sumPointsPrevMon', 'meduserstartPrevMon', 'minuserstartPrevMon', \
          'maxuserstartPrevMon', 'meanuserstartPrevMon']] = all_data.progress_apply(extract_compprevmonth_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [19:17<00:00, 280.75it/s]


**Create feature on most recent discussion, submissions and comments activity from the previous month**

In [139]:
def extract_otherprevmonth_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    #No of Comments
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    comm_user = comments[comments['UserID'] == user]
    
    comm_user_time = comm_user[comm_user['timestamp'] == prev_month]
    
    noCommPrevMon = len(comm_user_time)
    
    
    #submissions
    sub_user = submissions[submissions['UserID'] == user]
    
    sub_user_time = sub_user[sub_user['timestamp'] == prev_month]
    
    
    
    noSubStatsPrevMon = len(sub_user_time)
    
    SumSubFeatGPrevMon = sub_user_time['FeatureG'].sum()
    
    MeanSubFeatGPrevMon = sub_user_time['FeatureG'].mean()
    
    
    
    #discussions
    disc_user = discussions[discussions['UserID'] == user]
    
    #select previous month data
    
    disc_user_time = disc_user[disc_user['timestamp'] == prev_month]
    
    
    
    disc_user_time_group = disc_user_time.groupby(['UserID', 'timestamp'])['DiscID'].count()
    
    
    noDiscPrevMon = len(disc_user_time)
    
    
    SumDiscFeatFPrevMon = disc_user_time['FeatureF'].sum()
    
    MeanDiscFeatFPrevMon = disc_user_time['FeatureF'].mean()
    
    
    MinDiscPrevMon = disc_user_time_group.min()
    
    MeanDiscPrevMon = disc_user_time_group.mean()
    
    MedianDiscPrevMon = disc_user_time_group.median()
    
    MaxDiscPrevMon = disc_user_time_group.max()
    
    

    
    return pd.Series([noCommPrevMon, noSubStatsPrevMon, SumSubFeatGPrevMon, MeanSubFeatGPrevMon , noDiscPrevMon, SumDiscFeatFPrevMon, MeanDiscFeatFPrevMon,  MinDiscPrevMon, MeanDiscPrevMon, \
                     MedianDiscPrevMon, MaxDiscPrevMon])

In [140]:
all_data[['noCommPrevMon', 'noSubStatsPrevMon', 'SumSubFeatGPrevMon', 'MeanSubFeatGPrevMon' , 'noDiscPrevMon', 'SumDiscFeatFPrevMon', \
         'MeanDiscFeatFPrevMon',  'MinDiscPrevMon', 'MeanDiscPrevMon', 'MedianDiscPrevMon', 'MaxDiscPrevMon']] = \
all_data.progress_apply(extract_otherprevmonth_stats, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:58:33<00:00, 45.69it/s]


In [175]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,noSubStatsPrevMon,SumSubFeatGPrevMon,MeanSubFeatGPrevMon,noDiscPrevMon,SumDiscFeatFPrevMon,MeanDiscFeatFPrevMon,MinDiscPrevMon,MeanDiscPrevMon,MedianDiscPrevMon,MaxDiscPrevMon
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


**Adding supplementary information about Competitions Profile within user activity previous month**

In [176]:
def extract_comp_features_prevmon(df):
    
    year = df['year']
    
    month = df['month'] - 1
    
    user = df['UserID']
    
    
    #extract competitions possibly active at timeline of prediction, must have either started before prediction timeline or
    #will still be active at prediction time, endtime should be year of prediction or later
    
    comp = competitions[(year >= competitions['CompStartTime Year'])]
    
    
    #extract the time difference between prediction time & already started competions
    comp_delta_user = ((year - comp['CompStartTime Year']) * 12) + ((month - comp['CompStartTime Month'])) + 1
    
    
    #select competions happening on that month & year. Not all have end date, so can't use duration alone
    comp_1 = comp[(comp['CompStartTime Year'] == year) & (comp['CompStartTime Month'] == month)]
    
    #select competitions where the comp duration is within prediction timeline
    comp_2 = comp[comp['compDuration'] >= comp_delta_user]
    
    
    comp_3 = comp[comp['CompEndTime Year'] >= year]
    
    comp = pd.concat([comp_1, comp_2, comp_3])
    
    comp = comp[~comp.duplicated(keep = 'last')]
    
    

    
    TotalNoCompPrevMon = len(comp)
    
    
    return TotalNoCompPrevMon

In [177]:
all_data['TotalnoCompPrevMon'] = all_data.progress_apply(extract_comp_features_prevmon, axis = 1)

100%|██████████████████████████████████| 325055/325055 [19:09<00:00, 282.80it/s]


**Additional features on the ratio of disccussions, submissions, comments, competitions interacted with to no of competitions (user frequency of competitions interactions) - Previous Month**

In [178]:
all_data['PrevMon_disc_compratio'] = all_data['noDiscPrevMon']/all_data['TotalnoCompPrevMon']

all_data['PrevMon_sub_compratio'] = all_data['noSubStatsPrevMon']/all_data['TotalnoCompPrevMon']

all_data['PrevMon_comm_compratio'] = all_data['noCommPrevMon']/all_data['TotalnoCompPrevMon']

all_data['PrevMoncompInteractionratio'] = all_data['noCompPartPrevMon']/all_data['TotalnoCompPrevMon']


### 5.) The current user behaviour from the last 3 months is an indicator of next month behaviour, adding last 3 month feature statistics about supplementary datasets(Recency behaviour)

**Create feature on most recent competition participation from last 3 months**

In [179]:
all_data['timestamp'].iloc[0] - pd.DateOffset(months = 3)

Timestamp('2012-05-01 00:00:00')

**Create feature on most recent competition participation from the last 3 months**

In [180]:
def extract_compprevmonth3_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_3month = timestamp-pd.DateOffset(months = 3)
    
    
    #select previous month data
    
    comp_user_time = comp_user[(comp_user['timestamp'] <= prev_month) & (comp_user['timestamp'] >= prev_3month)]
    
    
    comp_user_time_group = comp_user_time.groupby(['UserID', 'timestamp'])['CompID'].count()
    
    
    #Rename to comppart
    #length of competitions
    
    noCompPartPrev3Mon = len(comp_user_time)
    
    noCompPartPrev3Monsec = comp_user_time['SecretCode'].sum()
    
    noCompPartPrev3Monnosec = noCompPartPrev3Mon - noCompPartPrev3Monsec
    
    
    #Need to add this
    MinCompPart3Mon = comp_user_time_group.min()
    
    MeanCompPart3Mon = comp_user_time_group.mean()
    
    MedianCompPart3Mon = comp_user_time_group.median()
    
    MaxCompPart3Mon = comp_user_time_group.max()
    
    
    minRankPrev3Mon = comp_user_time['PublicRank'].min()
    
    maxRankPrev3Mon = comp_user_time['PublicRank'].max()
    
    medRankPrev3Mon = comp_user_time['PublicRank'].median()
    
    meanRankPrev3Mon= comp_user_time['PublicRank'].mean()
    

    
    medsubCountPrev3Mon = comp_user_time['Successful Submission Count'].median()
    
    minsubCountPrev3Mon = comp_user_time['Successful Submission Count'].min()
    
    maxsubCountPrev3Mon = comp_user_time['Successful Submission Count'].max()
    
    meansubCountPrev3Mon = comp_user_time['Successful Submission Count'].mean()
    
    SumSubCountPrev3Mon = comp_user_time['Successful Submission Count'].sum()
    
    
    #points stats
    minPointsPrev3Mon = comp_user_time['Points Reward'].min()
    
    maxPointsPrev3Mon = comp_user_time['Points Reward'].max()
    
    meanPointsPrev3Mon = comp_user_time['Points Reward'].mean()
    
    medPointsPrev3Mon = comp_user_time['Points Reward'].median()
    
    
    #duration stats
    minDurationPrev3Mon = comp_user_time['compDuration'].min()
    
    maxDurationPrev3Mon = comp_user_time['compDuration'].max()
    
    meanDurationPrev3Mon = comp_user_time['compDuration'].mean()
    
    medDurationPrev3Mon = comp_user_time['compDuration'].median()
    
    
     #Kind
    
    SumKindPrev3Mon = comp_user_time['Kind'].sum()
    
    
    meduserstartPrev3Mon = comp_user_time['compuser_start'].median()
    
    minuserstartPrev3Mon = comp_user_time['compuser_start'].min()
    
    maxuserstartPrev3Mon = comp_user_time['compuser_start'].max()
    
    meanuserstartPrev3Mon = comp_user_time['compuser_start'].mean()
    

    
    
    
    return pd.Series([noCompPartPrev3Mon, noCompPartPrev3Monsec,  noCompPartPrev3Monnosec, minRankPrev3Mon, maxRankPrev3Mon, \
                      medRankPrev3Mon,  meanRankPrev3Mon, medsubCountPrev3Mon, minsubCountPrev3Mon,  \
                      maxsubCountPrev3Mon, meansubCountPrev3Mon, SumSubCountPrev3Mon, minPointsPrev3Mon, \
                      maxPointsPrev3Mon, meanPointsPrev3Mon, medPointsPrev3Mon, minDurationPrev3Mon , \
                      maxDurationPrev3Mon,  meanDurationPrev3Mon,  medDurationPrev3Mon, SumKindPrev3Mon, \
                      MinCompPart3Mon, MeanCompPart3Mon, MedianCompPart3Mon,  MaxCompPart3Mon,  meduserstartPrev3Mon, minuserstartPrev3Mon, \
                      maxuserstartPrev3Mon, meanuserstartPrev3Mon])

In [181]:
import gc
gc.collect()

0

In [182]:
all_data[['noCompPartPrev3Mon', 'noCompPartPrev3Monsec', 'noCompPartPrev3Monnosec', 'minRankPrev3Mon', 'maxRankPrev3Mon', \
          'medRankPrev3Mon',  'meanRankPrev3Mon', 'medsubCountPrev3Mon', 'minsubCountPrev3Mon',  \
          'maxsubCountPrev3Mon', 'meansubCountPrev3Mon', 'SumSubCountPrev3Mon', 'minPointsPrev3Mon', \
          'maxPointsPrev3Mon', 'meanPointsPrev3Mon', 'medPointsPrev3Mon', 'minDurationPrev3Mon' , 'maxDurationPrev3Mon', \
          'meanDurationPrev3Mon',  'medDurationPrev3Mon', 'SumKindPrev3Mon', \
          'MinCompPart3Mon', 'MeanCompPart3Mon', 'MedianCompPart3Mon',  'MaxCompPart3Mon',  'meduserstartPrev3Mon', \
          'minuserstartPrev3Mon', 'maxuserstartPrev3Mon', 'meanuserstartPrev3Mon']] = \
all_data.progress_apply(extract_compprevmonth3_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [23:09<00:00, 234.00it/s]


**Create feature on most recent discussion, submissions and comments activity from 3 months**

In [183]:
def extract_otherprevmonth3_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    #No of Comments
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_3month = timestamp-pd.DateOffset(months = 3)
    
    
    
    comm_user = comments[comments['UserID'] == user]
    
    comm_user_time = comm_user[(comm_user['timestamp'] <= prev_month) & (comm_user['timestamp'] >= prev_3month)]
    
    noCommPrev3Mon = len(comm_user_time)
    
    
    #submissions
    sub_user = submissions[submissions['UserID'] == user]
    
    sub_user_time = sub_user[(sub_user['timestamp'] <= prev_month) & (sub_user['timestamp'] >= prev_3month)]
    
    noSubStatsPrev3Mon = len(sub_user_time)
    
    SumSubFeatGPrev3Mon = sub_user_time['FeatureG'].sum()
    
    MeanSubFeatGPrev3Mon = sub_user_time['FeatureG'].mean()
    
    
    
    #discussions
    disc_user = discussions[discussions['UserID'] == user]
    
    #select previous month data
    
    disc_user_time = disc_user[(disc_user['timestamp'] <= prev_month) & (disc_user['timestamp'] >= prev_3month)]
    
    
    
    disc_user_time_group = disc_user_time.groupby(['UserID', 'timestamp'])['DiscID'].count()
    
    
    noDiscPrev3Mon = len(disc_user_time)
    
    
    SumDiscFeatFPrev3Mon = disc_user_time['FeatureF'].sum()
    
    MeanDiscFeatFPrev3Mon = disc_user_time['FeatureF'].mean()
    
    
    MinDiscPrev3Mon = disc_user_time_group.min()
    
    MeanDiscPrev3Mon = disc_user_time_group.mean()
    
    MedianDiscPrev3Mon = disc_user_time_group.median()
    
    MaxDiscPrev3Mon = disc_user_time_group.max()
    
    
    #points stats
    
    comp_user = compPart[compPart['UserID'] == user]

    #select previous month data
    
    comp_user_time = comp_user[(comp_user['timestamp'] <= prev_month) & (comp_user['timestamp'] >= prev_3month)]
    
    sumPointsPrev3Mon = comp_user_time['Points Reward'].sum()
    

    
    return pd.Series([noCommPrev3Mon, noSubStatsPrev3Mon, SumSubFeatGPrev3Mon, MeanSubFeatGPrev3Mon , noDiscPrev3Mon, SumDiscFeatFPrev3Mon, \
                      MeanDiscFeatFPrev3Mon,  MinDiscPrev3Mon, MeanDiscPrev3Mon, MedianDiscPrev3Mon, MaxDiscPrev3Mon, sumPointsPrev3Mon])

In [184]:
all_data[['noCommPrev3Mon', 'noSubStatsPrev3Mon', 'SumSubFeatGPrev3Mon', 'MeanSubFeatGPrev3Mon' , 'noDiscPrev3Mon', 'SumDiscFeatFPrev3Mon', \
         'MeanDiscFeatFPrev3Mon',  'MinDiscPrev3Mon', 'MeanDiscPrev3Mon', 'MedianDiscPrev3Mon', 'MaxDiscPrev3Mon', 'sumPointsPrev3Mon']] = \
all_data.progress_apply(extract_otherprevmonth3_stats, axis = 1)

100%|█████████████████████████████████| 325055/325055 [2:17:43<00:00, 39.34it/s]


In [185]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,SumSubFeatGPrev3Mon,MeanSubFeatGPrev3Mon,noDiscPrev3Mon,SumDiscFeatFPrev3Mon,MeanDiscFeatFPrev3Mon,MinDiscPrev3Mon,MeanDiscPrev3Mon,MedianDiscPrev3Mon,MaxDiscPrev3Mon,sumPointsPrev3Mon
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


### 5.) The current user behaviour from the last 6 months is an indicator of next month behaviour, adding last 6 month feature statistics about supplementary datasets(Recency behaviour)

In [186]:
def extract_compprevmonth6_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_6month = timestamp-pd.DateOffset(months = 6)
    
    
    #select previous month data
    
    comp_user_time = comp_user[(comp_user['timestamp'] <= prev_month) & (comp_user['timestamp'] >= prev_6month)]
    
    
    comp_user_time_group = comp_user_time.groupby(['UserID', 'timestamp'])['CompID'].count()
    
    
    #Rename to comppart
    #length of competitions
    
    noCompPartPrev6Mon = len(comp_user_time)
    
    noCompPartPrev6Monsec = comp_user_time['SecretCode'].sum()
    
    noCompPartPrev6Monnosec = noCompPartPrev6Mon - noCompPartPrev6Monsec
    
    
    #Need to add this
    MinCompPart6Mon = comp_user_time_group.min()
    
    MeanCompPart6Mon = comp_user_time_group.mean()
    
    MedianCompPart6Mon = comp_user_time_group.median()
    
    MaxCompPart6Mon = comp_user_time_group.max()
    
    
    minRankPrev6Mon = comp_user_time['PublicRank'].min()
    
    maxRankPrev6Mon = comp_user_time['PublicRank'].max()
    
    medRankPrev6Mon = comp_user_time['PublicRank'].median()
    
    meanRankPrev6Mon= comp_user_time['PublicRank'].mean()
    

    
    medsubCountPrev6Mon = comp_user_time['Successful Submission Count'].median()
    
    minsubCountPrev6Mon = comp_user_time['Successful Submission Count'].min()
    
    maxsubCountPrev6Mon = comp_user_time['Successful Submission Count'].max()
    
    meansubCountPrev6Mon = comp_user_time['Successful Submission Count'].mean()
    
    SumSubCountPrev6Mon = comp_user_time['Successful Submission Count'].sum()
    
    
    #points stats
    minPointsPrev6Mon = comp_user_time['Points Reward'].min()
    
    maxPointsPrev6Mon = comp_user_time['Points Reward'].max()
    
    meanPointsPrev6Mon = comp_user_time['Points Reward'].mean()
    
    medPointsPrev6Mon = comp_user_time['Points Reward'].median()
    
    
    #duration stats
    minDurationPrev6Mon = comp_user_time['compDuration'].min()
    
    maxDurationPrev6Mon = comp_user_time['compDuration'].max()
    
    meanDurationPrev6Mon = comp_user_time['compDuration'].mean()
    
    medDurationPrev6Mon = comp_user_time['compDuration'].median()
    
    
    #Kind
    
    SumKindPrev6Mon = comp_user_time['Kind'].sum()

    
    
    #CompStart
    meduserstartPrev6Mon = comp_user_time['compuser_start'].median()
    
    minuserstartPrev6Mon = comp_user_time['compuser_start'].min()
    
    maxuserstartPrev6Mon = comp_user_time['compuser_start'].max()
    
    meanuserstartPrev6Mon = comp_user_time['compuser_start'].mean()
    
    
    
    return pd.Series([noCompPartPrev6Mon, noCompPartPrev6Monsec,  noCompPartPrev6Monnosec, minRankPrev6Mon, maxRankPrev6Mon, \
                      medRankPrev6Mon,  meanRankPrev6Mon, medsubCountPrev6Mon, minsubCountPrev6Mon,  \
                      maxsubCountPrev6Mon, meansubCountPrev6Mon, SumSubCountPrev6Mon, minPointsPrev6Mon, \
                      maxPointsPrev6Mon, meanPointsPrev6Mon, medPointsPrev6Mon, minDurationPrev6Mon , \
                      maxDurationPrev6Mon,  meanDurationPrev6Mon,  medDurationPrev6Mon, SumKindPrev6Mon, \
                      MinCompPart6Mon, MeanCompPart6Mon, MedianCompPart6Mon,  MaxCompPart6Mon, \
                      meduserstartPrev6Mon, minuserstartPrev6Mon, maxuserstartPrev6Mon, meanuserstartPrev6Mon])

In [187]:
import gc
gc.collect()

0

In [188]:
all_data[['noCompPartPrev6Mon', 'noCompPartPrev6Monsec', 'noCompPartPrev6Monnosec', 'minRankPrev6Mon', 'maxRankPrev6Mon', \
          'medRankPrev6Mon',  'meanRankPrev6Mon', 'medsubCountPrev6Mon', 'minsubCountPrev6Mon',  \
          'maxsubCountPrev6Mon', 'meansubCountPrev6Mon', 'SumSubCountPrev6Mon', 'minPointsPrev6Mon', \
          'maxPointsPrev6Mon', 'meanPointsPrev6Mon', 'medPointsPrev6Mon', 'minDurationPrev6Mon' , 'maxDurationPrev6Mon', \
          'meanDurationPrev6Mon',  'medDurationPrev6Mon', 'SumKindPrev6Mon', 'MinCompPart6Mon', 'MeanCompPart6Mon', \
          'MedianCompPart6Mon',  'MaxCompPart6Mon', 'meduserstartPrev6Mon', 'minuserstartPrev6Mon', 'maxuserstartPrev6Mon', \
          'meanuserstartPrev6Mon']] = \
all_data.progress_apply(extract_compprevmonth6_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [23:22<00:00, 231.77it/s]


**Create feature on most recent discussion, submissions and comments activity from 6 months**

In [189]:
def extract_otherprevmonth6_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    #No of Comments
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_6month = timestamp-pd.DateOffset(months = 6)
    
    
    
    comm_user = comments[comments['UserID'] == user]
    
    comm_user_time = comm_user[(comm_user['timestamp'] <= prev_month) & (comm_user['timestamp'] >= prev_6month)]
    
    noCommPrev6Mon = len(comm_user_time)
    
    
    #submissions
    sub_user = submissions[submissions['UserID'] == user]
    
    sub_user_time = sub_user[(sub_user['timestamp'] <= prev_month) & (sub_user['timestamp'] >= prev_6month)]
    
    noSubStatsPrev6Mon = len(sub_user_time)
    
    SumSubFeatGPrev6Mon = sub_user_time['FeatureG'].sum()
    
    MeanSubFeatGPrev6Mon = sub_user_time['FeatureG'].mean()
    
    
    
    #discussions
    disc_user = discussions[discussions['UserID'] == user]
    
    #select previous month data
    
    disc_user_time = disc_user[(disc_user['timestamp'] <= prev_month) & (disc_user['timestamp'] >= prev_6month)]
    
    
    
    disc_user_time_group = disc_user_time.groupby(['UserID', 'timestamp'])['DiscID'].count()
    
    
    noDiscPrev6Mon = len(disc_user_time)
    
    
    SumDiscFeatFPrev6Mon = disc_user_time['FeatureF'].sum()
    
    MeanDiscFeatFPrev6Mon = disc_user_time['FeatureF'].mean()
    
    
    MinDiscPrev6Mon = disc_user_time_group.min()
    
    MeanDiscPrev6Mon = disc_user_time_group.mean()
    
    MedianDiscPrev6Mon = disc_user_time_group.median()
    
    MaxDiscPrev6Mon = disc_user_time_group.max()
    
    

    
    return pd.Series([noCommPrev6Mon, noSubStatsPrev6Mon, SumSubFeatGPrev6Mon, MeanSubFeatGPrev6Mon , noDiscPrev6Mon, SumDiscFeatFPrev6Mon, \
                      MeanDiscFeatFPrev6Mon,  MinDiscPrev6Mon, MeanDiscPrev6Mon, MedianDiscPrev6Mon, MaxDiscPrev6Mon])

In [190]:
all_data[['noCommPrev6Mon', 'noSubStatsPrev6Mon', 'SumSubFeatGPrev6Mon', 'MeanSubFeatGPrev6Mon' , 'noDiscPrev6Mon', 'SumDiscFeatFPrev6Mon', \
         'MeanDiscFeatFPrev6Mon',  'MinDiscPrev6Mon', 'MeanDiscPrev6Mon', 'MedianDiscPrev6Mon', 'MaxDiscPrev6Mon']] = \
all_data.progress_apply(extract_otherprevmonth6_stats, axis = 1)

100%|█████████████████████████████████| 325055/325055 [2:00:57<00:00, 44.79it/s]


In [191]:
import gc
gc.collect()

0

In [192]:
non_train = list(non_train)

In [194]:
train_data = all_data[all_data['tag'] == 'train']

In [195]:
train_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,noSubStatsPrev6Mon,SumSubFeatGPrev6Mon,MeanSubFeatGPrev6Mon,noDiscPrev6Mon,SumDiscFeatFPrev6Mon,MeanDiscFeatFPrev6Mon,MinDiscPrev6Mon,MeanDiscPrev6Mon,MedianDiscPrev6Mon,MaxDiscPrev6Mon
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,2.0,1.0,0.5,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


### 6) Measure the last activity status as an indication of user last active duration, Starting from the precious month: When was the user last active, what has been the duration from last active, extracting user activity duration statistics for overall time, 3 mon and 6 mon time windows.

**Extracting user last active overall**

In [196]:
def extract_user_last_active(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    joinmonth = df['is_joinmonth']
    
    
    #Select user, pick previous month
    train_user = train_data[train_data['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    

    
    prev_mon_lags = train_user[train_user['timestamp'] <= prev_month]
    
    prev_mon_lags = prev_mon_lags.sort_values(['timestamp'], ascending = False)
    
    
    #due to test set instance, prev mon should be last month with user target data
    
    try:
        prev_month = prev_mon_lags['timestamp'].head(1).values[0]
        
    except:
        
        prev_month = np.nan
    
    
    
    #new user, no last active
    if user in non_train:
        
        return np.nan
    
    
    #new user just at join month
    elif joinmonth == 1:
        
        return 0
    
    
    #previous month was active, 0 no of days since last active
    elif prev_mon_lags[prev_mon_lags['timestamp'] == prev_month]['Target'].values[0] == 1:
        
        return 0

    
    #user has never been active, subtract current date from since inactivity date. Add more weight for users with inactive constant
    elif prev_mon_lags['Target'].sum() < 1:
        
        no_positive = prev_mon_lags['timestamp'].tail(1).values[0].astype(str)
            
        total_days = pd.to_datetime(prev_month) - pd.to_datetime(no_positive)
        
        return int(total_days.days/30) + len(prev_mon_lags)
    
        
    #user active history, calculate last active from most recent active against previous month
    
    else:
        
        last_positive = prev_mon_lags[prev_mon_lags['Target'] == 1]['timestamp'].head(1).values[0].astype(str)
        
        total_days = pd.to_datetime(prev_month) - pd.to_datetime(last_positive)
        
        return int(total_days.days/30)

In [197]:
import gc
gc.collect()

0

In [198]:
all_data['num_days_last_active'] = all_data.progress_apply(extract_user_last_active, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:18:09<00:00, 69.32it/s]


In [199]:
all_data['num_days_last_active'].head()

0    7.0
1    5.0
2    6.0
3    8.0
4    9.0
Name: num_days_last_active, dtype: float64

In [200]:
all_data.groupby(['num_days_last_active'])['Target'].mean().head(10)

num_days_last_active
0.0    0.441885
1.0    0.124780
2.0    0.085763
3.0    0.059963
4.0    0.053694
5.0    0.040002
6.0    0.036820
7.0    0.030248
8.0    0.030688
9.0    0.026797
Name: Target, dtype: float64

**Extracting user last active 6mon**

In [201]:
def extract_user_last_active_6mon(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    joinmonth = df['is_joinmonth']
    
    
    train_user = train_data[train_data['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_6month = timestamp-pd.DateOffset(months = 6)
    

    
    prev_mon_lags = train_user[(train_user['timestamp'] <= prev_month) & (train_user['timestamp'] >= prev_6month)]
    
    prev_mon_lags = prev_mon_lags.sort_values(['timestamp'], ascending = False)
    
    
    #due to test set instance, prev mon should be last month with user target data
    
    try:
        prev_month = prev_mon_lags['timestamp'].head(1).values[0]
        
    except:
        
        prev_month = np.nan
    
    
    
    #new user, no last active
    if user in non_train:
        
        return np.nan
    
    
    #new user just at join month
    elif joinmonth == 1:
        
        return 0
    
    
    #previous month was active, 0 no of days since last active
    elif prev_mon_lags[prev_mon_lags['timestamp'] == prev_month]['Target'].values[0] == 1:
        
        return 0

    
    #user has never been active, subtract current date from since inactivity date. Add more weight for users with inactive constant
    elif prev_mon_lags['Target'].sum() < 1:
        
        no_positive = prev_mon_lags['timestamp'].tail(1).values[0].astype(str)
            
        total_days = pd.to_datetime(prev_month) - pd.to_datetime(no_positive)
        
        return int(total_days.days/30) + len(prev_mon_lags)
    
        
    #user active history, calculate last active from most recent active against previous month
    
    else:
        
        last_positive = prev_mon_lags[prev_mon_lags['Target'] == 1]['timestamp'].head(1).values[0].astype(str)
        
        total_days = pd.to_datetime(prev_month) - pd.to_datetime(last_positive)
        
        return int(total_days.days/30)

In [202]:
import gc
gc.collect()

0

In [203]:
all_data['num_days_last_active_6mon'] = all_data.progress_apply(extract_user_last_active_6mon, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:17:52<00:00, 69.57it/s]


In [204]:
all_data.groupby(['num_days_last_active_6mon'])['Target'].mean().tail(10)

num_days_last_active_6mon
0.0     0.441885
1.0     0.124780
2.0     0.085763
3.0     0.059963
4.0     0.053694
5.0     0.040002
6.0     0.130435
7.0     0.022831
9.0     0.020829
11.0    0.034078
Name: Target, dtype: float64

**Extracting user active duration status: How long has a user sustained actviivty**

In [205]:
def extract_user_active_duration(df):
    
    user = df['UserID']
    
    timestamp = df['timestamp']
    
    joinmonth = df['is_joinmonth']
    
    histmonths = df['HistMonths']
    
    
    
    train_user = train_data[train_data['UserID'] == user]
    
    train_user = train_user.sort_values(['timestamp'])
    
    
    #initial assignment
    user_min_active = np.nan
    
    user_max_active = np.nan
        
    user_median_active = np.nan  
    
    user_mean_active = np.nan
    
    user_sum_active = np.nan
    
    
    #new user, no last active
    if user in non_train:
        
        pass
    
    
    #new user just at join month
    elif joinmonth == 1:
        
        pass
    
    
    else:
        
        user_duration = list()
    
        train_user_neg = train_user[(train_user['timestamp'] < timestamp) & (train_user['Target'] == 0)]
        
        
        if (len(train_user_neg) > 1):
            
            for index in range(1, len(train_user_neg)):
                
                duration = int((train_user_neg['timestamp'].iloc[index] - train_user_neg['timestamp'].\
                                      iloc[index-1]).days/30)
                
                user_duration.append(duration)
                
            user_min_active = np.min(user_duration)
    
            user_max_active = np.max(user_duration)
        
            user_median_active = np.median(user_duration)    
    
            user_mean_active = np.mean(user_duration)
    
            user_sum_active = np.sum(user_duration)
                
            
        else:
            
                    
            user_min_active = histmonths - len(train_user_neg)
    
            user_max_active = histmonths - len(train_user_neg)
        
            user_median_active = histmonths - len(train_user_neg)
    
            user_mean_active = histmonths - len(train_user_neg)
    
            user_sum_active = histmonths - len(train_user_neg)
                
   
    
    return pd.Series([user_min_active, user_max_active, user_median_active, user_mean_active, \
                      user_sum_active])

In [206]:
all_data[['user_min_active', 'user_max_active', 'user_median_active', 'user_mean_active', \
           'user_sum_active']] = all_data.progress_apply(extract_user_active_duration, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:16:57<00:00, 70.39it/s]


In [207]:
all_data[['user_min_active', 'user_max_active', 'user_median_active', 'user_mean_active', \
          'user_sum_active']].head()

,user_min_active,user_max_active,user_median_active,user_mean_active,user_sum_active
0,0.0,3.0,1.0,1.071429,15.0
1,1.0,1.0,1.0,1.000000,4.0
2,1.0,1.0,1.0,1.000000,5.0
3,0.0,3.0,1.0,1.066667,16.0
4,0.0,3.0,1.0,1.062500,17.0


In [208]:
import gc
gc.collect()

0

**Create additional time window-Quarter**

In [211]:
all_data['quarter'] = np.floor(abs(all_data['month'])/3)

In [212]:
all_data['quarter'] = all_data['quarter'].replace(4, 3)

all_data.loc[all_data['month'] == 3, 'quarter'] = 0

all_data.loc[all_data['month'] == 6, 'quarter'] = 1

all_data.loc[all_data['month'] == 9, 'quarter'] = 2

In [213]:
all_data.groupby(['quarter'])['Target'].mean()

quarter
0.0    0.182377
1.0    0.167385
2.0    0.102877
3.0    0.130099
Name: Target, dtype: float64

In [214]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,MedianDiscPrev6Mon,MaxDiscPrev6Mon,num_days_last_active,num_days_last_active_6mon,user_min_active,user_max_active,user_median_active,user_mean_active,user_sum_active,quarter
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,7.0,11.0,0.0,3.0,1.0,1.071429,15.0,2.0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,5.0,5.0,1.0,1.0,1.0,1.000000,4.0,2.0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,6.0,11.0,1.0,1.0,1.0,1.000000,5.0,2.0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,8.0,11.0,0.0,3.0,1.0,1.066667,16.0,2.0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,NaN,NaN,9.0,11.0,0.0,3.0,1.0,1.062500,17.0,3.0


### 7) Measuring apparent changes in user behaviour- How often has behaviour remained positive, negative, changed from positive to negative, negative to positive for overall period

In [217]:
def hist_diff_activity(df):
    
    prev_month = df['timestamp'] - pd.DateOffset(months = 1)
    
    user = df['UserID']
    
    train_user = all_data[all_data['UserID'] == user]
    
    
    #select all past years data
    train_user_time = train_user[(train_user['timestamp'] <= prev_month)]
    
    train_user_time = train_user_time.sort_values(['year', 'month'], ascending = False)
    
    #Restricting to historical no of months with Target
    train_user_time = train_user_time[train_user_time['Target'].isnull() == False]
    
    all_diff = list()
    
    targs = train_user_time['Target']
    
    
    for i in range(0, len(targs)):
        
        if i != (len(targs) - 1):
                         
            diff = targs.iloc[i] -  targs.iloc[i + 1]
        
            if (diff == 0) & (targs.iloc[i] == 1):
            
                all_diff.append('1_1')
                
                
            elif (diff == 0) & (targs.iloc[i] == 0):
                
                all_diff.append('0_0')
        
            
            elif diff == 1:
            
                all_diff.append('0_1')
                
                
            elif diff == -1:
                
                all_diff.append('1_0')
    
    
    if len(all_diff) >= 1:
        
        pos_pos_behav = all_diff.count('1_1')
    
        neg_neg_behav = all_diff.count('0_0')
    
        neg_pos_behav = all_diff.count('0_1')
        
        pos_neg_behav = all_diff.count('1_0')
    
        
        
    else:
        
        pos_pos_behav = np.nan
        
        neg_neg_behav = np.nan
        
        neg_pos_behav = np.nan
        
        pos_neg_behav = np.nan
    
    
    
    
    
    
    return pd.Series([pos_pos_behav, neg_neg_behav, neg_pos_behav, pos_neg_behav])
    

In [218]:
all_data[['pos_pos_behav', 'neg_neg_behav', 'neg_pos_behav', 'pos_neg_behav']] = \
all_data.progress_apply(hist_diff_activity, axis = 1)

100%|█████████████████████████████████| 325055/325055 [1:37:43<00:00, 55.44it/s]


In [237]:
all_data.groupby('neg_pos_behav')['Target'].mean()

neg_pos_behav
0.0     0.054849
1.0     0.132342
2.0     0.195186
3.0     0.227551
4.0     0.226521
5.0     0.268173
6.0     0.279133
7.0     0.275362
8.0     0.241379
9.0     0.250000
10.0    0.000000
Name: Target, dtype: float64

### 8) Extracting statistics about already started user competitions- Previous month, 3 mon and 6 mon time windows. Users are more likely to continue already started competitions depending on their rank, submission counts, potential points.

In [220]:
def extract_compprevmonth_active_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    year = df['year']
    
    month = df['month']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    
    #select previous month data
    
    comp_user_time = comp_user[comp_user['timestamp'] == prev_month]
    
    
    comp_1 = comp_user_time[(comp_user_time['CompEndTime Year'] > year)]
    
    comp_2 = comp_user_time[(comp_user_time['CompEndTime Year'] == year) & \
                            (comp_user_time['CompEndTime Month'] >= month)]
    
    
    comp = pd.concat([comp_1, comp_2])
    
    comp = comp[~comp.duplicated(keep = 'last')]
    
    
    
    #no of competitions
    
    noCompPartPrevMonActive = len(comp)
    
    
    
    #Rank Statistics
    sumRankPrevMonActive = comp['PublicRank'].sum()
    
    minRankPrevMonActive = comp['PublicRank'].min()
    
    maxRankPrevMonActive = comp['PublicRank'].max()
    
    medRankPrevMonActive = comp['PublicRank'].median()
    
    meanRankPrevMonActive = comp['PublicRank'].mean()
    
    
    
    #Submission Statistics
    sumSubCountPrevMonActive = comp['Successful Submission Count'].sum()

    medsubCountPrevMonActive = comp['Successful Submission Count'].median()
    
    minsubCountPrevMonActive = comp['Successful Submission Count'].min()
    
    maxsubCountPrevMonActive = comp['Successful Submission Count'].max()
    
    meansubCountPrevMonActive = comp['Successful Submission Count'].mean()
    
    
    
    #points stats
    minPointsPrevMonActive = comp['Points Reward'].min()
    
    maxPointsPrevMonActive = comp['Points Reward'].max()
    
    meanPointsPrevMonActive = comp['Points Reward'].mean()
    
    medPointsPrevMonActive = comp['Points Reward'].median()
    
    sumPointsPrevMonActive = comp['Points Reward'].sum()
    
    
    
    
    
    return pd.Series([noCompPartPrevMonActive, sumRankPrevMonActive, minRankPrevMonActive, maxRankPrevMonActive, \
                     medRankPrevMonActive, meanRankPrevMonActive, sumSubCountPrevMonActive, medsubCountPrevMonActive, \
                      minsubCountPrevMonActive, maxsubCountPrevMonActive, meansubCountPrevMonActive, \
                     minPointsPrevMonActive, maxPointsPrevMonActive, meanPointsPrevMonActive, \
                      medPointsPrevMonActive, sumPointsPrevMonActive])

In [221]:
all_data[['noCompPartPrevMonActive', 'sumRankPrevMonActive', 'minRankPrevMonActive', 'maxRankPrevMonActive', \
          'medRankPrevMonActive', 'meanRankPrevMonActive', 'sumSubCountPrevMonActive', 'medsubCountPrevMonActive', \
          'minsubCountPrevMonActive', 'maxsubCountPrevMonActive', 'meansubCountPrevMonActive', \
          'minPointsPrevMonActive', 'maxPointsPrevMonActive', 'meanPointsPrevMonActive', \
          'medPointsPrevMonActive', 'sumPointsPrevMonActive']] = \
all_data.progress_apply(extract_compprevmonth_active_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [29:03<00:00, 186.43it/s]


In [222]:
def extract_compprev3month_active_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    year = df['year']
    
    month = df['month']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_3month = timestamp - pd.DateOffset(months = 3)
    
    
    #select previous month data
    
    comp_user_time = comp_user[(comp_user['timestamp'] <= prev_month) & (comp_user['timestamp'] >= prev_3month)]
    
    
    comp_1 = comp_user_time[(comp_user_time['CompEndTime Year'] > year)]
    
    comp_2 = comp_user_time[(comp_user_time['CompEndTime Year'] == year) & \
                            (comp_user_time['CompEndTime Month'] >= month)]
    
    
    comp = pd.concat([comp_1, comp_2])
    
    comp = comp[~comp.duplicated(keep = 'last')]
    
    
    
    #no of competitions
    
    noCompPartPrev3MonActive = len(comp)
    
    
    
    #Rank Statistics
    sumRankPrev3MonActive = comp['PublicRank'].sum()
    
    minRankPrev3MonActive = comp['PublicRank'].min()
    
    maxRankPrev3MonActive = comp['PublicRank'].max()
    
    medRankPrev3MonActive = comp['PublicRank'].median()
    
    meanRankPrev3MonActive = comp['PublicRank'].mean()
    
    
    
    #Submission Statistics
    sumSubCountPrev3MonActive = comp['Successful Submission Count'].sum()

    medsubCountPrev3MonActive = comp['Successful Submission Count'].median()
    
    minsubCountPrev3MonActive = comp['Successful Submission Count'].min()
    
    maxsubCountPrev3MonActive = comp['Successful Submission Count'].max()
    
    meansubCountPrev3MonActive = comp['Successful Submission Count'].mean()
    
    
    
    #points stats
    minPointsPrev3MonActive = comp['Points Reward'].min()
    
    maxPointsPrev3MonActive = comp['Points Reward'].max()
    
    meanPointsPrev3MonActive = comp['Points Reward'].mean()
    
    medPointsPrev3MonActive = comp['Points Reward'].median()
    
    sumPointsPrev3MonActive = comp['Points Reward'].sum()
    
    
    
    
    
    return pd.Series([noCompPartPrev3MonActive, sumRankPrev3MonActive, minRankPrev3MonActive, maxRankPrev3MonActive, \
                     medRankPrev3MonActive, meanRankPrev3MonActive, sumSubCountPrev3MonActive, medsubCountPrev3MonActive, \
                      minsubCountPrev3MonActive, maxsubCountPrev3MonActive, meansubCountPrev3MonActive, \
                     minPointsPrev3MonActive, maxPointsPrev3MonActive, meanPointsPrev3MonActive, \
                      medPointsPrev3MonActive, sumPointsPrev3MonActive])

In [223]:
all_data[['noCompPartPrev3MonActive', 'sumRankPrev3MonActive', 'minRankPrev3MonActive', 'maxRankPrev3MonActive', \
          'medRankPrev3MonActive', 'meanRankPrev3MonActive', 'sumSubCountPrev3MonActive', 'medsubCountPrev3MonActive', \
          'minsubCountPrev3MonActive', 'maxsubCountPrev3MonActive', 'meansubCountPrev3MonActive', \
          'minPointsPrev3MonActive', 'maxPointsPrev3MonActive', 'meanPointsPrev3MonActive', \
          'medPointsPrev3MonActive', 'sumPointsPrev3MonActive']] = \
all_data.progress_apply(extract_compprev3month_active_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [30:06<00:00, 179.95it/s]


In [224]:
def extract_compprev6month_active_stats(df):
    
    timestamp = df['timestamp']
    
    user = df['UserID']
    
    year = df['year']
    
    month = df['month']
    
    
    comp_user = compPart[compPart['UserID'] == user]
    
    
    prev_month = timestamp - pd.DateOffset(months = 1)
    
    prev_6month = timestamp - pd.DateOffset(months = 6)
    
    
    #select previous month data
    
    comp_user_time = comp_user[(comp_user['timestamp'] <= prev_month) & (comp_user['timestamp'] >= prev_6month)]
    
    
    comp_1 = comp_user_time[(comp_user_time['CompEndTime Year'] > year)]
    
    comp_2 = comp_user_time[(comp_user_time['CompEndTime Year'] == year) & \
                            (comp_user_time['CompEndTime Month'] >= month)]
    
    
    comp = pd.concat([comp_1, comp_2])
    
    comp = comp[~comp.duplicated(keep = 'last')]
    
    
    
    #no of competitions
    
    noCompPartPrev6MonActive = len(comp)
    
    
    
    #Rank Statistics
    sumRankPrev6MonActive = comp['PublicRank'].sum()
    
    minRankPrev6MonActive = comp['PublicRank'].min()
    
    maxRankPrev6MonActive = comp['PublicRank'].max()
    
    medRankPrev6MonActive = comp['PublicRank'].median()
    
    meanRankPrev6MonActive = comp['PublicRank'].mean()
    
    
    
    #Submission Statistics
    sumSubCountPrev6MonActive = comp['Successful Submission Count'].sum()

    medsubCountPrev6MonActive = comp['Successful Submission Count'].median()
    
    minsubCountPrev6MonActive = comp['Successful Submission Count'].min()
    
    maxsubCountPrev6MonActive = comp['Successful Submission Count'].max()
    
    meansubCountPrev6MonActive = comp['Successful Submission Count'].mean()
    
    
    
    #points stats
    minPointsPrev6MonActive = comp['Points Reward'].min()
    
    maxPointsPrev6MonActive = comp['Points Reward'].max()
    
    meanPointsPrev6MonActive = comp['Points Reward'].mean()
    
    medPointsPrev6MonActive = comp['Points Reward'].median()
    
    sumPointsPrev6MonActive = comp['Points Reward'].sum()
    
    
    
    
    
    return pd.Series([noCompPartPrev6MonActive, sumRankPrev6MonActive, minRankPrev6MonActive, maxRankPrev6MonActive, \
                     medRankPrev6MonActive, meanRankPrev6MonActive, sumSubCountPrev6MonActive, medsubCountPrev6MonActive, \
                      minsubCountPrev6MonActive, maxsubCountPrev6MonActive, meansubCountPrev6MonActive, \
                     minPointsPrev6MonActive, maxPointsPrev6MonActive, meanPointsPrev6MonActive, \
                      medPointsPrev6MonActive, sumPointsPrev6MonActive])

In [225]:
all_data[['noCompPartPrev6MonActive', 'sumRankPrev6MonActive', 'minRankPrev6MonActive', 'maxRankPrev6MonActive', \
          'medRankPrev6MonActive', 'meanRankPrev6MonActive', 'sumSubCountPrev6MonActive', 'medsubCountPrev6MonActive', \
          'minsubCountPrev6MonActive', 'maxsubCountPrev6MonActive', 'meansubCountPrev6MonActive', \
          'minPointsPrev6MonActive', 'maxPointsPrev6MonActive', 'meanPointsPrev6MonActive', \
          'medPointsPrev6MonActive', 'sumPointsPrev6MonActive']] = \
all_data.progress_apply(extract_compprev6month_active_stats, axis = 1)

100%|██████████████████████████████████| 325055/325055 [30:13<00:00, 179.29it/s]


In [226]:
all_data.head()

,UserID,month,year,CompPart,Comment,Sub,Disc,Target,tag,FeatureX,...,sumSubCountPrev6MonActive,medsubCountPrev6MonActive,minsubCountPrev6MonActive,maxsubCountPrev6MonActive,meansubCountPrev6MonActive,minPointsPrev6MonActive,maxPointsPrev6MonActive,meanPointsPrev6MonActive,medPointsPrev6MonActive,sumPointsPrev6MonActive
0,ID_XI7BAR4Y,8,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,ID_XI7BAR4Y,8,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,ID_XI7BAR4Y,9,2,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,ID_XI7BAR4Y,9,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,ID_XI7BAR4Y,10,3,0.0,0.0,0.0,0.0,0.0,train,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [227]:
all_data.to_pickle('Final Data Prep.pkl', compression = 'gzip')

## The extracted dataframe for modelling expected future user behavior utilizes-

1) User historical behaviour up till timeline of prediction. (Historical user behaviour/Observed user behaviour)

2) Statistics about on-going competitions (Nature of competitions, characteristics of the competitions)

3) User information and characterisitics available on the platform

4) User activity from the previous month

5) User activity(Specific level detail) in the last 3 months

6) User activity(Specific level detail) in the last 6 months